# Initializing QTLab stuff

In [1]:
%run D:\qtlab_replacement\init.py
%matplotlib qt4
import matplotlib.pyplot as plt
from instruments import *
import numpy as np
import sweep
import pandas as pd
from save_pkl import *

In [2]:
pna = Agilent_N5242A('pna', address = 'PNA')
pxa = Agilent_N9030A('pxa', address = 'PXA')
lo1 = Agilent_E8257D('lo1', address = 'PSG1')
awg_tek = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')

root: WARNING  Tektronix_AWG5014 : get all not yet fully functional


In [3]:
#DAC Говоркова 14бит, 500MS, 1.5V на 50 ом?
#Выдаёт TRIG на SYNC0 и CLK на С3 для daq0
#Если виснет наглухо, нажать в USB Tree View "Restart device"
#Если делает фигню вместо сигнала (бывает часто), то просто перезапустить ядро и всё остальное
#Каналы DAC Говоркова
awg_ex = awg_tek
awg_ex_I_ch = 1
awg_ex_Q_ch = 2
#Каналы Tektronix
awg_ro = awg_tek
awg_ro_I_ch = 3
awg_ro_Q_ch = 4

In [96]:
#Включаем всё что надо и выключаем всё что не надо
awg_ch_on = [awg_ro_I_ch, awg_ro_Q_ch]
awg_ch_off = [ch_id for ch_id in range(7) if ch_id not in awg_ch_on]

for ch_id in awg_ch_off:
    ch_settings = awg.get_channel_settings(ch_id)
    ch_settings['on'] = False
    awg.set_channel_settings(ch_id, ch_settings)

for ch_id in awg_ch_on:
    ch_settings = awg.get_channel_settings(ch_id)
    ch_settings['on'] = True
    ch_settings['slow'] = 0
    ch_settings['mode'] = 'ram'
    ch_settings['delay'] = 0
    awg.set_channel_settings(ch_id, ch_settings)
#хз что
awg.ProgTimer(awg.intToBytes([0xfc, 0]))
#1 триггер на один период сигнала
awg.set_trigger_repeats(1)
#Выход триггера
awg.set_trigger_pulse("SYNC0", awg.create_delayed_trigger_pulse(1.) )
awg.set_trigger_pulse("SYNC3", awg.create_delayed_trigger_pulse(1.) )

In [97]:
awg_tek.set_ch1_amplitude(2)
awg_tek.set_ch2_amplitude(2)

True

In [98]:
# Источник тока - в autorange
#current.set_autorange(1)
#Мощности гетеродинов, постоянные
#Мощность гетеродина для возбуждения 13-16 дБм
lo_ex_pow = 14
lo1.set_status(0)
lo1.set_power(lo_ex_pow)
#pna как lo2 (excitation)
#Port1 - выход
#Мощность гетеродина для считывания 10-13 дБм, с учётом разветвителя 13-16 dBm
lo_ro_pow = 16
pna.write("OUTP ON")
pna.write("SOUR1:POW1:MODE ON")
pna.write("SOUR1:POW2:MODE OFF")
pna.set_sweep_mode("CW")
pna.set_power(lo_ro_pow)

True

In [99]:
#Частоты считывания и возбуждения
ex_freq = 6.611e9

ro_freq =  9.525e9 #ph
#ro_freq = 7.595e9#amp
#ro_freq = 7.6009e9
#ro_freq = 7.6024e9#cent


#Промежуточные частоты, постоянны.
ro_if = 62.5e6
ex_if = 125e6

#Амплитуда I/Q сигналов с awg
#Максимальная амплитуда awg 8000, дальше искажения, теоретически 8192
#8000 = 750 mV на 50 ом = 10.5 dbm
#Смесители имеют 1 дБ компрессию при 5 дБм = 4234, так что больше лучше не ставить
awg_amp_ro = 5000
awg_amp_ex = 8000
#Задержка сигнала *2e-9
zero_delay = 38

#Чачтоты гетеродина
ex_lo_freq = ex_freq + ex_if
ro_lo_freq = ro_freq + ro_if

In [100]:
#DAQ карта Spectrum M3i2132, daq0
#Внешний триггер, внешнее тактирование от awg
SampleRate_ro = 500e6
SampleRate_ex = 1000e6
M3i2132_init(daq0, SampleRate_ro)
awg_tek.set_clock(SampleRate_ex)

root: WARNING  Tektronix_AWG5014 : Clock set to 1000000000.0. This is not fully functional yet. To avoid problems, it is better not to change the clock during operation


True

#Heterodyne Calibrate IQ mixers

In [101]:
#Калибровка смесителя

#Настройки смесителя
#O - offset по амплитуде (находится при калибровке)
#P - фазовый offset (находится при калибровке)
#F - промежуточная частота 
#A - амплитуда в разрядах DAC
#synced - некая синхронизация, хз
execfile('IQCalibrate.py')

target = "both"

awg.set_repetition_period(3e-6)
awg_tek.set_repetition_period(2e-6)
awg_tek.run()

if target == "both" or target == "ex":
    lo1.set_frequency(ex_lo_freq)
    lo1.set_status(1)
    awg_ex.set_trigger_mode('CONT')
    awg_ex.run()
    #Функция меняет период повторения AWG. Если чё.
    settings_ex, D2_estimates, snr_last_ex = IQCalibrate2(pxa, awg_ex, ex_lo_freq, awg_amp_ex, ex_if, 1.0e-6, \
                                            awg_ex_I_ch, awg_ex_Q_ch, adaptive=True, sample_period = 1./SampleRate_ex)
    awg_ex.set_trigger_mode('TRIG')
if target == "both" or target == "ro":
    #Калибровка считывания
    pna.set_cw_freq(ro_lo_freq)
    #Функция меняет период повторения AWG. Если чё.
    execfile('IQCalibrate.py')
    print (ro_if, ro_lo_freq)
    settings_ro, D2_estimates, snr_last_ro = IQCalibrate2(pxa, awg_ro, ro_lo_freq, awg_amp_ro, ro_if, 1.0e-6, \
                                            awg_ro_I_ch, awg_ro_Q_ch, adaptive=True, sample_period = 1./SampleRate_ro)

125000000.0
{1: {'A': 8000, 'F': 125000000.0, 'O': 0, 'delay': 0, 'P': 0, 'synced': True, 'O0': 0}, 2: {'A': 8000, 'F': 125000000.0, 'O': 0, 'delay': 0, 'P': -1.5707963267948966, 'synced': True, 'O0': 0}}
[ 0.08975827  0.09004875  0.08869433]
{1: {'A': 8000, 'F': 125000000.0, 'O': 0, 'delay': 0, 'P': 0, 'synced': True, 'O0': 0}, 2: {'A': 8000, 'F': 125000000.0, 'O': 0, 'delay': 0, 'P': -0.52359877559829915, 'synced': True, 'O0': 0}}
[ 0.09032169  0.09032125  0.08820986]
{1: {'A': 8000, 'F': 125000000.0, 'O': 0, 'delay': 0, 'P': 0, 'synced': True, 'O0': 0}, 2: {'A': 8000, 'F': 125000000.0, 'O': 0, 'delay': 0, 'P': 0.5235987755982987, 'synced': True, 'O0': 0}}
[ 0.0908809   0.09021071  0.08935112]
{1: {'A': 8000, 'F': 125000000.0, 'O': 0, 'delay': 0, 'P': 0, 'synced': True, 'O0': 0}, 2: {'A': 8000, 'F': 125000000.0, 'O': 0, 'delay': 0, 'P': 1.5707963267948966, 'synced': True, 'O0': 0}}
[ 0.09080539  0.0911336   0.08830724]
{1: {'A': 8000, 'F': 125000000.0, 'O': 0, 'delay': 0, 'P': 0, 'sy

KeyboardInterrupt: 

In [11]:
#print settings_ex
print settings_ex, snr_last_ex
print settings_ro, snr_last_ro

{1: {'A': 8000, 'F': 125000000.0, 'O': -188.15390006423186, 'delay': 0, 'P': 0, 'synced': False, 'O0': -261.0}, 2: {'A': 4917.9776324873246, 'F': 125000000.0, 'O': -163.10312113041067, 'delay': 7.497691464625429e-12, 'P': 1.7936870179749409, 'synced': False, 'O0': -211.02661615735582}} 2937.93264862
{3: {'A': 3512.8287479208343, 'F': 62500000.0, 'O': 111.34580603835488, 'delay': 0, 'P': 0, 'synced': False, 'O0': 42}, 4: {'A': 5000, 'F': 62500000.0, 'O': 277.19365247151728, 'delay': -6.3405555029086825e-10, 'P': 4.1239368022250344, 'synced': False, 'O0': 208.55474991810465}} 133.339470624


In [9]:
#Load settings_excite
(ex_lo_freqs, settings) = load_pkl('settings_ex')
settings_ex = settings[0]

In [10]:
#Load settings_ro
(ro_lo_freqs, settings) = load_pkl('settings_ro')
settings_ro = settings[0]

In [12]:
#Save settings_excite
execfile("save_pkl.py")
save_pkl_noplt('settings_ex', ( np.array([ex_lo_freq]) ,np.array([settings_ex]) ) )

In [ ]:
#print settings_ro
print settings_ro, snr_last_ro

In [13]:
#Save settings_ro
save_pkl_noplt('settings_ro', ( np.array([ro_lo_freq]) ,np.array([settings_ro]) ) )

##Input by hands

In [ ]:
settings_ex     =  \
{3: {'A': 5000, 'F': 62500000.0, 'O': -382.06005044462881, 'delay': 0, 'P': 0, 'synced': False, 'O0': -423}, \
 4: {'A': 4943.1176879565164, 'F': 62500000.0, 'O': -375.48577298767844, 'delay': 5.9971491857426073e-10, 'P': 4.6867918711326118, 'synced': False, 'O0': -292.13135238498415}} 
settings_ro = \
{0: {'A': 2057.0411273316768, 'F': 62500000.0, 'O': 144, 'delay': 0, 'P': 0, 'synced': False, 'O0': 174}, \
 5: {'A': 5000, 'F': 62500000.0, 'O': 13, 'delay': -1.1514683006298718e-10, 'P': 1.6149928736604171, 'synced': False, 'O0': 34}} 
#settings_excite = \
#{3: {'A': 2756.7900590837767, 'F': 62500000.0, 'O': 298, 'delay': 0, 'P': 0, 'synced': False, 'O0': 316.0}, \
# 4: {'A': 3000, 'F': 62500000.0, 'O': 263.0, 'delay': -6.7240040453484868e-11, 'P': 1.4246290879731895, 'synced': False, 'O0': 266.0}}

#Finding the resonator

In [25]:
pulse_length = 20e-6
PulsePeriod = 10e-6

awg_ex.set_trigger_mode('CONT')
exPulseI = [settings_ex[awg_ex_I_ch]['O']]+RectSine(settings_ex[awg_ex_I_ch]['A'], 1./ex_if, pulse_length, 
                    settings_ex[awg_ex_I_ch]['P'], settings_ex[awg_ex_I_ch]['O'], 0, sample_period = 1./SampleRate_ex)+[settings_ex[awg_ex_I_ch]['O']]

exPulseQ = [settings_ex[awg_ex_Q_ch]['O']]+RectSine(settings_ex[awg_ex_Q_ch]['A'], 1./ex_if, pulse_length, 
                        settings_ex[awg_ex_Q_ch]['P'], settings_ex[awg_ex_Q_ch]['O'], 0, sample_period = 1./SampleRate_ex)+[settings_ex[awg_ex_Q_ch]['O']]

lo1.set_frequency(ex_lo_freq)

awg_ex.LoadRAM(awg_ex_I_ch, exPulseI)
awg_ex.LoadRAM(awg_ex_Q_ch, exPulseQ)

In [ ]:
pulse_length = 10e-6
PulsePeriod = 20e-6

roPulseI = RectSine(settings_ro[awg_ro_I_ch]['A'], 1./ro_if, pulse_length, 
                    settings_ro[awg_ro_I_ch]['P'], settings_ro[awg_ro_I_ch]['O'], 0, sample_period = 1./SampleRate_ro)

roPulseQ = RectSine(settings_ro[awg_ro_Q_ch]['A'], 1./ro_if, pulse_length, 
                    settings_ro[awg_ro_Q_ch]['P'], settings_ro[awg_ro_Q_ch]['O'], 0, sample_period = 1./SampleRate_ro)

plt.plot(roPulseQ)
pna.set_cw_freq(ro_lo_freq)

#roPulseI = np.ones((32768,),dtype=int)*0
#roPulseQ = np.ones((32768,),dtype=int)*1000

awg.LoadRAM(awg_ro_I_ch, roPulseI)
awg.LoadRAM(awg_ro_Q_ch, roPulseQ)

In [ ]:
lo1.set_status(0)

In [ ]:
pxa.set_centerfreq(9.5e9)
pxa.set_span(2e9)
pxa.set_nop(10001)
plt.plot(pxa.get_tracedata()[0], pxa.get_tracedata()[1])

In [66]:
#Диапазон частот
#ro_freqs = np.linspace(7.593e9,7.613e9, 500)
ro_freqs = np.linspace(ro_freq-20e6,ro_freq+20e6, 201)
#Число усреднений
nums = 5000
#Длина импульса и период повторения
ro_pulse_length =4e-6
PulsePeriod = 4e-6
#Диапазон DAQ, мВ
daq_lenght = ro_pulse_length
Range = 50
#Скорость DAQ
SampleRate = 500e6
software_averages = 5

#Настраиваем awg
awg.set_repetition_period(PulsePeriod)
awg.set_trigger_pulse("SYNC0", awg.create_delayed_trigger_pulse(1.) )

#AWG500 IQ pulse
#roPulseI = RectSine(awg_amp, 1./ro_if, ro_pulse_length, 0, 0, 0)
#roPulseQ = RectSine(awg_amp, 1./ro_if, ro_pulse_length, np.pi/2., 0, 0)
roPulseI = RectSine(settings_ro[awg_ro_I_ch]['A']/3., 1./ro_if, ro_pulse_length, 
                    settings_ro[awg_ro_I_ch]['P'], settings_ro[awg_ro_I_ch]['O'], 0, sample_period = 1./SampleRate_ro)

roPulseQ = RectSine(settings_ro[awg_ro_Q_ch]['A']/3., 1./ro_if, ro_pulse_length, 
                        settings_ro[awg_ro_Q_ch]['P'], settings_ro[awg_ro_Q_ch]['O'], 0, sample_period = 1./SampleRate_ro)
roPulseI[0]=roPulseI[-1]
roPulseQ[0]=roPulseQ[-1]
awg_tek.set_numpoints(int(PulsePeriod/2e-9)-1000)
awg_ro.LoadRAM(awg_ro_I_ch, roPulseI)
awg_ro.LoadRAM(awg_ro_Q_ch, roPulseQ)

#Размер выборки DAQ
signal_size = int(np.ceil(daq_lenght*SampleRate))
segsize = 2*int( 2**np.ceil(np.log2(signal_size)) )
print segsize
posttrigger = segsize*3/4
signal_start = segsize - posttrigger 
signal_end = segsize - posttrigger + signal_size + zero_delay*2
if signal_end>segsize:
    signal_end = segsize

#Настраиваем DAQ
daq0.stop()
M3i2132_init(daq0, SampleRate)
#daq0.init_channel0_multiple_recording(nums=nums, segsize=segsize, posttrigger=posttrigger, amp=amp, offs=offs)
daq0.init_channel01_multiple_recording(rate=SampleRate, nums=nums, segsize=segsize, 
                                  posttrigger=posttrigger, amp0=Range, amp1=Range)
#Предыдущая функция работает через жопу и как-то сбрасывает 50 ом, поэтому включаем заново
daq0.trigger_termination_50Ohm()
daq0.buffer_setup()
segsize = signal_end - signal_start
t1=time.time()

#Контейнеры для данных
measQ = np.zeros((len(ro_freqs), segsize))
measI = np.zeros((len(ro_freqs), segsize))
Amp = np.zeros( len(ro_freqs) )
Ph = np.zeros( len(ro_freqs) )

#Функции для демодуляции
t = linspace(0.,segsize/SampleRate, segsize, endpoint=False)
dm_sin = sin(2.*pi*ro_if*t)
dm_cos = cos(2.*pi*ro_if*t)

#Графики
plt.figure("Amplitude")
amp_plot = plt.plot(ro_freqs, Amp)[0]
plt.figure("Phase")
ph_plot = plt.plot(ro_freqs, Ph)[0]

#Долбим квадратный синус с разными частотами и мерием, что проходит
try:
    for i, freq in enumerate(ro_freqs):
        pna.set_cw_freq(ro_lo_freq)
        for software_average_id in range(software_averages):
            #Мерием по триггеру
            daq0.start_with_trigger_and_waitready()
            #Усредняем результат
            daq_buf = daq0.readout_doublechannel_multimode_bin()
            measI[i] = np.sum(daq_buf[0], axis=0)[signal_start:signal_end]
            measQ[i] = np.sum(daq_buf[1], axis=0)[signal_start:signal_end]
            measI[i] -=np.mean(measI[i])
            measQ[i] -=np.mean(measQ[i])
            #Делаем преобразованеи до нулевой частоты 
            I = np.mean(measI[i]*dm_cos - measQ[i]*dm_sin)
            Q = np.mean(measQ[i]*dm_cos + measI[i]*dm_sin)
            
            #Амплитуа
            Amp[i] = np.sqrt(I**2 + Q**2)/nums
            #Фаза
            Ph[i] = np.arctan2(I,Q)
            
            daq0.stop()
        #Рисуем графики
        plt.figure("Amplitude")
        amp_plot.set_data(ro_freqs[:i+1], Amp[:i+1])
        plt.ylim((0., np.max(Amp[:i+1])))
        plt.figure("Phase")
        ph_plot.set_data(ro_freqs[:i+1], Ph[:i+1])
        plt.ylim((np.min(Ph[:i+1]), np.max(Ph[:i+1])))
        plt.pause(0.01)
        
except KeyboardInterrupt:pass
finally:pass
print "Execution time: ",time.time()-t1," s"

C:\Python27\Lib\site-packages\matplotlib\axes\_base.py:2809: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=-2.12426749204, top=-2.12426749204
  'bottom=%s, top=%s') % (bottom, top))
C:\Python27\Lib\site-packages\matplotlib\backend_bases.py:2399: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


4096
Execution time:  468.475999832  s


In [ ]:
plt.figure("measI")
plt.pcolormesh(np.asarray(measI));
plt.colorbar()
plt.figure("measQ")
plt.pcolormesh(np.asarray(measQ))
plt.colorbar()
plt.figure("measabs")
plt.pcolormesh(np.asarray(measQ)**2+np.asarray(measI)**2)
plt.colorbar()

In [ ]:
plt.figure("IQ")
plt.plot(I)
plt.plot(Q)

In [ ]:
plt.plot(roPulseI)

In [ ]:
%matplotlib qt4

In [ ]:
plt.figure()
plt.plot(np.mean(np.fft.fft(np.asarray(measI)), axis=0))
plt.plot(np.mean(np.fft.fft(np.asarray(measQ)), axis=0))

In [ ]:
Amp_ref = Amp
Ph_ref = Ph

In [ ]:
plt.figure()
plt.plot(ro_freqs,Amp)
plt.figure()
plt.plot(ro_freqs,Ph)

In [ ]:
#Unwrap phase
dPh = np.diff(Ph)
for i in range(len(dPh)):
    if np.abs(dPh[i]) >= np.pi/2.:
        if dPh[i] >0:
            dPh[i] = dPh[i] - np.pi
        elif dPh[i] <0:
            dPh[i] = dPh[i] + np.pi
plt.figure()
Ph = np.cumsum( np.hstack([Ph[0], dPh]) )

In [ ]:
dPh = np.diff(Ph)
plt.figure("Delay")
plt.plot(ro_freqs [0:-1], dPh)
plt.ylim((np.min(dPh), np.max(dPh)))

In [ ]:
plt.figure()
plt.plot(ro_freqs, Ph)

In [ ]:
plt.figure("Phase")
ph_plot.set_data(ro_freqs[:i+1], Ph[:i+1])
plt.ylim((np.min(Ph[:i+1]), np.max(Ph[:i+1])))
plt.pause(0.01)

In [ ]:
#Ищет резонатор по минимуму
#Но лучше найти точку ручками и написать
#Оптимальная точка для считывания не обязательно минимум
#new_ro_freq = ro_freqs[np.argmax(np.abs(Amp))]
#print 'Old resonator frequency was {0:3.5g}'.format(ro_freq)
#print 'New resonator frequency is {0:3.5g}'.format(new_ro_freq)
old_ro_freq = ro_freq
ro_freq = 9.5253e9
print ro_freq
ro_lo_freq = ro_freq + ro_if

In [ ]:
plt.figure()
plt.plot(measI[0])

In [ ]:
#Save 
save_pkl_noplt('search_of_resonator', {"Amp": Amp , "Ph": Ph, "ro_freqs": ro_freqs } )

In [ ]:
#Load
pkl = load_pkl('resonator_pulse')
Amp_res = pkl["Amp"]
Ph_res = pkl["Ph"]
ro_freqs = pkl["ro_freqs"]

In [ ]:
plt.figure()
plt.plot(ro_freqs, Amp_res)


# Finding the qubit

In [73]:
ex_freqs = arange(ex_freq-50e6, ex_freq+50e6, 0.5e6)

nums = 1000
Range = 50
SampleRate = 500e6

ro_pulse_length = 3e-6
pulse_length = 9e-6
PulsePeriod = 10e-6
daq_length = 5e-6

software_averages = 20

exPulseI = RectSine(settings_ex[awg_ex_I_ch]['A'], 1./ex_if, pulse_length, 
                    settings_ex[awg_ex_I_ch]['P'], settings_ex[awg_ex_I_ch]['O'], 0, sample_period = 1./SampleRate_ex)

exPulseQ = RectSine(settings_ex[awg_ex_Q_ch]['A'], 1./ex_if, pulse_length, 
                        settings_ex[awg_ex_Q_ch]['P'], settings_ex[awg_ex_Q_ch]['O'], 0, sample_period = 1./SampleRate_ex)

roPulseI = RectSine(settings_ro[awg_ro_I_ch]['A'], 1./ro_if, ro_pulse_length, 
                    settings_ro[awg_ro_I_ch]['P'], settings_ro[awg_ro_I_ch]['O'], 2e-6+500e-9, sample_period = 1./SampleRate_ro)

roPulseQ = RectSine(settings_ro[awg_ro_Q_ch]['A'], 1./ro_if, ro_pulse_length, 
                        settings_ro[awg_ro_Q_ch]['P'], settings_ro[awg_ro_Q_ch]['O'], 2e-6+500e-9, sample_period = 1./SampleRate_ro)
exPulseI[0]=exPulseI[-1] 
exPulseQ[0]=exPulseQ[-1]

#Настраиваем awg
awg.set_repetition_period(PulsePeriod)
awg_tek.set_repetition_period(PulsePeriod-1e-6)
awg.set_trigger_pulse("SYNC0", awg.create_delayed_trigger_pulse(0.) )
awg.set_trigger_pulse("SYNC3", awg.create_delayed_trigger_pulse(0.) )

awg_ex.LoadRAM(awg_ex_I_ch, exPulseI)
awg_ex.LoadRAM(awg_ex_Q_ch, exPulseQ)

#awg_tek.set_numpoints(int(PulsePeriod/2e-9))
awg_ro.LoadRAM(awg_ro_I_ch, roPulseI)
awg_ro.LoadRAM(awg_ro_Q_ch, roPulseQ)
awg_tek.run()

pna.set_cw_freq(ro_lo_freq)

#Настраиваем DAQ
signal_size = int(np.ceil(daq_length*SampleRate))
segsize = 2*int( 2**np.ceil(np.log2(signal_size)) )
print segsize
posttrigger = segsize*3/4
signal_start = segsize - posttrigger 
signal_end = segsize - posttrigger + signal_size + zero_delay*2

daq0.stop()
M3i2132_init(daq0, SampleRate)
daq0.init_channel01_multiple_recording(rate=SampleRate, nums=nums, segsize=segsize, 
                                  posttrigger=posttrigger, amp0=Range, amp1=Range)
#Предыдущая функция работает через жопу и как-то сбрасывает 50 ом, поэтому включаем заново
daq0.trigger_termination_50Ohm()
daq0.buffer_setup()

lo1.set_status(1)

segsize = signal_end-signal_start
#Контейнеры для данных
measQ = np.zeros((len(ex_freqs), segsize))
measI = np.zeros((len(ex_freqs), segsize))

Amp = np.zeros( len(ex_freqs) )
Ph = np.zeros( len(ex_freqs) )

#Функции для демодуляции
t = linspace(0.,segsize/SampleRate, segsize, endpoint=False)

dm_sin = sin(2.*pi*ro_if*t)
dm_cos = cos(2.*pi*ro_if*t)

#Графики
plt.figure("Amplitude")
amp_plot = plt.plot(ex_freqs, Amp)[0]
plt.figure("Phase")
ph_plot = plt.plot(ex_freqs, Ph)[0]

t1=time.time()
try:
    for i, freq in enumerate(ex_freqs):
        lo1.set_frequency(freq - ex_if)
        for software_average_id in range(software_averages):
            #Мерием по триггеру
            daq0.start_with_trigger_and_waitready()
            #Усредняем результат
            daq_buf = daq0.readout_doublechannel_multimode_bin()
            measI[i] += np.sum(daq_buf[0], axis=0)[signal_start:signal_end]
            measQ[i] += np.sum(daq_buf[1], axis=0)[signal_start:signal_end]
            daq0.stop()
            
        #Делаем преобразованеи до нулевой частоты 
        I = np.mean(measI[i]*dm_cos - measQ[i]*dm_sin)
        Q = np.mean(measQ[i]*dm_cos + measI[i]*dm_sin)
                
        #Амплитуа
        Amp[i] = np.sqrt(I**2 + Q**2)/nums
        #Фаза
        Ph[i] = np.arctan(I/Q)
        
        #Рисуем графики
        plt.figure("Amplitude")
        amp_plot.set_data(ex_freqs[:i+1], Amp[:i+1])
        plt.ylim((0., np.max(Amp[:i+1])))
        plt.figure("Phase")
        ph_plot.set_data(ex_freqs[:i+1], Ph[:i+1])
        plt.ylim((np.min(Ph[:i+1]), np.max(Ph[:i+1])))
        plt.pause(0.01)

except KeyboardInterrupt:pass
print time.time()-t1

8192
306.474999905


C:\Python27\Lib\site-packages\matplotlib\axes\_base.py:2809: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=-0.216118768688, top=-0.216118768688
  'bottom=%s, top=%s') % (bottom, top))


In [89]:
awg.set_trigger_repeats(1)
awg.set_trigger_period(0)

In [41]:
new_ex_freq = ex_freqs[np.argmin(Amp)]
print 'Old excitation frequency was {0:3.5g}'.format(ex_freq)
print 'New excitation frequency is {0:3.5g}'.format(new_ex_freq)
old_ex_freq = ex_freq
ex_freq = new_ex_freq

Old excitation frequency was 6.4005e+09
New excitation frequency is 6.3945e+09


In [ ]:
#Save 
save_pkl_noplt('qubit_pulse_1', {"Amp": Amp , "Ph": Ph, "ex_freqs": ex_freqs,
                        "ex_freq":ex_freq,"ro_freq": ro_freq} )

# Pulsed measurements

## Delay measurement

In [13]:
SampleRate = 500e6
nums = 10000
Range = 50
software_averages = 10
ro_pulse_length = 1e-6
PulsePeriod = 10e-6

awg.set_trigger_pulse("SYNC0", awg.create_delayed_trigger_pulse(1.) )

roPulseI = RectSine(settings_ro[awg_ro_I_ch]['A'], 1./ro_if, ro_pulse_length, 
                    settings_ro[awg_ro_I_ch]['P'], settings_ro[awg_ro_I_ch]['O'], 0)

roPulseQ = RectSine(settings_ro[awg_ro_Q_ch]['A'], 1./ro_if, ro_pulse_length, 
                        settings_ro[awg_ro_Q_ch]['P'], settings_ro[awg_ro_Q_ch]['O'], 0)

awg.LoadRAM(awg_ro_I_ch, roPulseI)
awg.LoadRAM(awg_ro_Q_ch, roPulseQ)
awg_ex.set_trigger_mode('TRIG')
    
pna.set_cw_freq(ro_lo_freq)

#Настраиваем DAQ
segsize = int(2**np.ceil(np.log(ro_pulse_length*SampleRate)/np.log(2))*2)*2
print segsize
posttrigger = segsize*3/4

measI = np.zeros( segsize )

daq0.stop()
M3i2132_init(daq0, SampleRate)
daq0.init_channel0_multiple_recording(nums=nums, segsize=segsize, posttrigger=posttrigger, amp=Range)
#daq0.init_channel01_multiple_recording(rate=SampleRate, nums=nums, segsize=segsize, 
#                                  posttrigger=posttrigger, amp0=Range, amp1=Range)
daq0.trigger_termination_50Ohm()
daq0.buffer_setup()

lo1.set_status(0)

for software_average_id in range(software_averages):
    daq0.start_with_trigger_and_waitready()
    measI += np.sum(daq0.readout_singlechannel_multimode_bin()[0], axis=0)
    #measI = np.sum(daq0.readout_doublechannel_multimode_bin()[0],axis=0)
    daq0.stop()

measI = measI - np.mean(measI)
plt.figure()
plt.plot(measI)
#plt.plot(roPulseI-np.mean(roPulseI))
threshold = np.max(np.abs(measI)[0:10])*2
zero_delay = np.argwhere(abs(measI)>threshold)[0][0] - (segsize - posttrigger)
print "Pulse delay = ", zero_delay

2048
Pulse delay =  15


In [33]:
zero_delay = 13

# Incoherent decay

In [66]:
#Задержка считывания после прекращения возбуждения
ro_delays = np.arange(-2e-6, 2e-6, 0.1e-6)

ro_pulse_length = 2e-6
posttrig_t = 1e-6
ex_pulse_length = 5e-6

nums = 1000
Range = 50
SampleRate = 500e6

PulsePeriod = 50e-6

software_averages = 10

#Настраиваем частоты
lo1.set_frequency(ex_lo_freq)
pna.set_cw_freq(ro_lo_freq)

#Настраиваем awg
data_len = int(PulsePeriod/2e-9)+1
awg.set_repetition_period(PulsePeriod)
awg_tek.set_repetition_period(PulsePeriod-1e-6)
awg_ex.set_trigger_mode('TRIG')
max_ro_delay = np.max(ro_delays)

#Инициализация DAQ
signal_size = int(np.ceil( (ro_pulse_length+posttrig_t) * SampleRate ))
hardware_segsize = 2*int( 2**np.ceil(np.log2(signal_size+int((max_ro_delay+ex_pulse_length)*SampleRate))) )

posttrigger = hardware_segsize*3/4
signal_start = int((max_ro_delay+ex_pulse_length)*SampleRate) + hardware_segsize - posttrigger + zero_delay
signal_end = signal_start + signal_size
if signal_end>hardware_segsize:
    signal_end = hardware_segsize
    
daq0.stop()
daq0.set_spc_samplerate(SampleRate)
daq0.init_channel01_multiple_recording(rate=SampleRate, nums=nums, segsize=hardware_segsize, 
                                  posttrigger=posttrigger, amp0=Range, amp1=Range)
daq0.trigger_termination_50Ohm()
daq0.buffer_setup()

lo1.set_status(1)
segsize = signal_end -  signal_start
print hardware_segsize, posttrigger, int(max_ro_delay*SampleRate), signal_start, signal_end
#Контейнеры для данных
measQ = np.zeros((len(ro_delays), segsize))
measI = np.zeros((len(ro_delays), segsize))
I = np.zeros( len(ro_delays) )
Q = np.zeros( len(ro_delays) )

#Функции для демодуляции
t = linspace(0.,segsize/SampleRate, segsize, endpoint=False)
dm_sin = sin(2.*pi*ro_if*t)
dm_cos = cos(2.*pi*ro_if*t)

#Графики
plt.figure("I")
amp_plot = plt.plot(ro_delays, I)[0]
plt.figure("Q")
ph_plot = plt.plot(ro_delays, Q)[0]

roPulseI = RectSine(settings_ro[awg_ro_I_ch]['A'], 1./ro_if, ro_pulse_length, 
                    settings_ro[awg_ro_I_ch]['P'], settings_ro[awg_ro_I_ch]['O'], 
                    max_ro_delay + ex_pulse_length, sample_period = 1./SampleRate_ro)
    
roPulseQ = RectSine(settings_ro[awg_ro_Q_ch]['A'], 1./ro_if, ro_pulse_length, 
                    settings_ro[awg_ro_Q_ch]['P'], settings_ro[awg_ro_Q_ch]['O'],
                    max_ro_delay + ex_pulse_length, sample_period = 1./SampleRate_ro)
roPulseI [0] = roPulseI [-1]
roPulseQ[0] = roPulseQ [-1]

awg_ro.LoadRAM(awg_ro_I_ch, roPulseI)
awg_ro.LoadRAM(awg_ro_Q_ch, roPulseQ)

# setting trigger to start readout after excitation pulses
awg.set_trigger_pulse("SYNC0", awg.create_delayed_trigger_pulse(1.) )
awg.set_trigger_pulse("SYNC3", awg.create_delayed_trigger_pulse(1) )

for i, ro_delay in enumerate(ro_delays):
    exPulseI = RectSine(settings_ex[awg_ex_I_ch]['A'], 1./ex_if, ex_pulse_length,
                        settings_ex[awg_ex_I_ch]['P'], settings_ex[awg_ex_I_ch]['O'], 
                        max_ro_delay - ro_delay, sample_period = 1./SampleRate_ex)
    exPulseQ = RectSine(settings_ex[awg_ex_Q_ch]['A'], 1./ex_if, ex_pulse_length, 
                        settings_ex[awg_ex_Q_ch]['P'], settings_ex[awg_ex_Q_ch]['O'], 
                        max_ro_delay - ro_delay, sample_period = 1./SampleRate_ex)

    #Программируем импульсы в awg
    awg_ex.LoadRAM(awg_ex_I_ch, exPulseI[0:data_len])
    awg_ex.LoadRAM(awg_ex_Q_ch, exPulseQ[0:data_len])
    
    for software_average_id in range(software_averages):
        #Мерием по триггеру
        daq0.start_with_trigger_and_waitready()
        #Усредняем результат
        measI[i] += np.sum(daq0.readout_doublechannel_multimode_bin()[0],
                           axis=0)[signal_start:signal_end]
        measQ[i] += np.sum(daq0.readout_doublechannel_multimode_bin()[1],
                           axis=0)[signal_start:signal_end]
        measI[i] -=np.mean(measI[i]) 
        measQ[i] -=np.mean(measQ[i])
        daq0.stop()
        
    #Делаем преобразованеи до нулевой частоты 
    I[i] = np.mean(measI[i]*dm_cos - measQ[i]*dm_sin)/(nums*software_averages)
    Q[i] = np.mean(measQ[i]*dm_cos + measI[i]*dm_sin)/(nums*software_averages)
                
    plt.figure("I")
    amp_plot.set_data(ro_delays[:i+1], I[:i+1])
    plt.ylim((np.min(I[:i+1]), np.max(I[:i+1])))
    
    plt.figure("Q")
    ph_plot.set_data(ro_delays[:i+1], Q[:i+1])
    plt.ylim((np.min(Q[:i+1]), np.max(Q[:i+1])))
    plt.pause(0.01)

16384 12288 949 7583 9083


C:\Python27\Lib\site-packages\matplotlib\axes\_base.py:2809: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=-0.0111645093609, top=-0.0111645093609
  'bottom=%s, top=%s') % (bottom, top))
C:\Python27\Lib\site-packages\matplotlib\axes\_base.py:2809: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=0.0113479185633, top=0.0113479185633
  'bottom=%s, top=%s') % (bottom, top))


In [67]:
plt.figure("rawI")
plt.pcolormesh(np.asarray(measI).T)
plt.figure("rawQ")
plt.pcolormesh(np.asarray(measQ).T)

In [28]:
I = np.mean((np.asarray(measI)*dm_cos - np.asarray(measQ)*dm_sin)[:, 6681:7200], axis=1)
Q = np.mean((np.asarray(measI)*dm_sin + np.asarray(measQ)*dm_cos)[:, 6681:7200], axis=1)
plt.figure()
plt.plot(I)
plt.figure()
plt.plot(Q)

In [44]:
plt.figure()
plt.pcolormesh(signalI)
plt.figure()
plt.pcolormesh(signalQ)

In [59]:
fit_range = ro_delays[ro_delays>0]

# фитует результаты измерений экспонентой
f = lambda x, A0 ,A1, t: A0 + A1*np.exp(-x/t)

fit_res={'I':zeros(3),'Q':zeros(3), 'Ierr': zeros(3), 'Qerr': zeros(3)}

for dat,id in zip([I,Q],['I','Q']):
    A0 = dat[-1]
    A1 = dat[0]
    t = np.mean(ro_delays)
    (res, err) = so.curve_fit(f, ro_delays, dat, p0 = (A0, A1, t))
    fit_res[id+'err'] = np.sqrt(np.diag(err))
    fit_res[id] = res
    plt.figure(id)
    plt.plot(ro_delays,  dat, 'o')
    plt.plot(ro_delays,  f(ro_delays, *tuple(res) ),
         label = "T1={:.2f}us".format(res[2]*1e6), lw = 2, color = 'red')
    plt.legend()
print fit_res 

{'I': array([  1.11310912e-01,   6.85639197e-03,   1.68565169e-05]), 'Q': array([  2.05782211e-01,  -7.50012935e-04,   1.96417112e-06]), 'Qerr': array([  4.65941460e-05,   2.21531009e-04,   9.65436202e-07]), 'Ierr': array([  1.04327883e-03,   9.38192762e-04,   4.42283168e-06])}


In [60]:
A0r = I[-1]
A0i = Q[-1]
A1r = I[0]
A1i = Q[0]
positive_delays =  ro_delays[ro_delays>=0]
positive_delays_S21 = I[ro_delays>=0]+1j*Q[ro_delays>=0]

f = lambda x,A0r,Aoi,A1r,A1i,t: (A0r+A0i*1j+((A1r-A0r)+(A1i-A0i)*1j)*np.exp(-x/t))
cost = lambda p: np.abs(f(positive_delays,*p)-positive_delays_S21)**2
t = np.mean(ro_delays)
fitresults = so.leastsq(cost, (A0r,A0i,A1r,A1i,t))
plt.plot(ro_delays,  I, 'o', color='blue')
plt.plot(ro_delays,  Q, 'o', color='green')
plt.plot(ro_delays,  f(ro_delays, *(fitresults[0])).real,
    label = "T1={:.2f}us".format(fitresults[0][4]*1e6), lw = 2, color = 'blue')
plt.plot(ro_delays,  f(ro_delays, *(fitresults[0])).imag, lw = 2, color = 'green')
fitted_curve = (positive_delays, np.log10(np.abs(f(positive_delays, *(fitresults[0]))))*10, np.angle(f(positive_delays, *(fitresults[0]))))
plt.legend()

In [61]:
#Save with plot
header = {'date':datetime.datetime.now(), 'type': 'Incoherent decay', 'name': 'Decay Fliegenklatsche second cooldown', \
          'ro_pulse_length':ro_pulse_length, 'ex_pulse_length':ex_pulse_length, \
          'ex_freq':ex_freq,'ro_freq': ro_freq}
measurement = ((ro_delays, np.asarray([0]), np.log10(np.abs(I+1j*Q))*10, np.angle(I+1j*Q)))
annotation='Decay time: $T_1={0:3.2f} \mu s$'.format(fitresults[0][4]*1e6)
save_pkl(header, measurement, annotation=annotation, curve_fit=fitted_curve)

In [39]:
#Save 
save_pkl_noplt('T1_1', {"I": I , "Q": Q, "fit_res": fit_res, 
                "ro_delays": ro_delays, "fit_func": "lambda x, A0 ,A1, t: A0 + A1*np.exp(-x/t)",
                "ex_freq":ex_freq,"ro_freq": ro_freq} )

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<ipython-input-39-c52c68ec3570> in <module>()
      3 save_pkl(header, 'T1_1', {"I": I , "Q": Q, "fit_res": fitresults, 
      4                 "ro_delays": ro_delays, "fit_func": "lambda x, A0 ,A1, t: A0 + A1*np.exp(-x/t)",
----> 5                 "ex_freq":ex_freq,"ro_freq": ro_freq} )

c:\qtlab\ReiData\qtlab_addons\scripts\save_pkl.py in save_pkl(header, data, plot, curve_fit, annotation)
     47 
     48         if ( (len(data) == 4) and plot):
---> 49                 if (len(data[2].shape) == 1):
     50                         plt.figure()
     51                         plt.plot(data[0], data[2])

AttributeError: 'str' object has no attribute 'shape'


In [161]:
pkl = load_pkl('cernter_freq/inc_dec3')
Amp = pkl["Amp"]
Ph = pkl["Ph"]
ro_delays = pkl["ro_delays"]

In [147]:
30.*2e-9

6.000000000000001e-08

# Rabi

In [15]:
#Длительности возбуждающих импульсов
pulse_lengths = np.arange(0., 2e-6, 4e-9)

PulsePeriod = 20e-6

ro_pulse_length = 1e-6
#Длительность выборки DAQ
SampleRate = 500e6
nums = 1000
Range = 50
SampleRate_ro = SampleRate
SampleRate_ex = SampleRate*2
software_averages = 10

lo1.set_frequency(ex_lo_freq)
pna.set_cw_freq(ro_lo_freq)

awg.set_repetition_period(PulsePeriod)
awg_tek.set_repetition_period(PulsePeriod-1e-6)

#Настраиваем awg
data_len_ex = int(PulsePeriod*SampleRate_ex)+1 
data_len_ro = int(PulsePeriod*SampleRate_ro)+1 
awg.set_repetition_period(PulsePeriod)
awg_ex.set_trigger_mode('TRIG')

#Инициализация DAQ
signal_size = int(np.ceil((np.max(pulse_lengths)+ro_pulse_length)*SampleRate_ro))
segsize = 2*int( 2**np.ceil(np.log2(signal_size)) )
print segsize
posttrigger = segsize*3/4
signal_start = segsize - posttrigger 
signal_end = segsize - posttrigger + signal_size + zero_delay*2
if signal_end>segsize:
    signal_end = segsize
#segsize = 2048
#posttrigger = 512*3
    
daq0.stop()
daq0.set_spc_samplerate(SampleRate)
daq0.init_channel01_multiple_recording(rate=SampleRate, nums=nums, segsize=segsize, 
                                  posttrigger=posttrigger, amp0=Range, amp1=Range)
daq0.trigger_termination_50Ohm()
daq0.buffer_setup()


lo1.set_status(1)
#segsize = signal_end -  signal_start
#Контейнеры для данных
measQ = np.zeros((len(pulse_lengths), segsize))
measI = np.zeros((len(pulse_lengths), segsize))
I_Rabi = np.zeros( len(pulse_lengths) )
Q_Rabi = np.zeros( len(pulse_lengths) )

#Функции для демодуляции
t = linspace(0.,segsize/SampleRate, segsize, endpoint=False)
dm_sin = sin(2.*pi*ro_if*t)
dm_cos = cos(2.*pi*ro_if*t)

#Графики
plt.figure("I_Rabi")
amp_plot = plt.plot(pulse_lengths, I_Rabi)[0]
plt.figure("Q_Rabi")
ph_plot = plt.plot(pulse_lengths, Q_Rabi)[0]

max_pulse_length = np.max(pulse_lengths)

#Считывающий импульс сразу после возбуждающего
roPulseI = RectSine(settings_ro[awg_ro_I_ch]['A'], 1./ro_if, ro_pulse_length,
                    settings_ro[awg_ro_I_ch]['P'], settings_ro[awg_ro_I_ch]['O'], max_pulse_length+500e-9, sample_period = 1./SampleRate_ro)
roPulseQ = RectSine(settings_ro[awg_ro_Q_ch]['A'], 1./ro_if, ro_pulse_length, 
                    settings_ro[awg_ro_Q_ch]['P'], settings_ro[awg_ro_Q_ch]['O'], max_pulse_length+500e-9, sample_period = 1./SampleRate_ro)
awg_ro.LoadRAM(awg_ro_I_ch, roPulseI)
awg_ro.LoadRAM(awg_ro_Q_ch, roPulseQ)
# setting trigger to start readout immideately after excitation puls
# zero_delay надо сначала померить в delay synchronization
#triggerPulse = [0, zero_delay+int(max_pulse_length/1e-9), 3, 100, 3, 1, 0, 4]
#SyncPulseProg = awg.intToBytes(triggerPulse)
#awg.ProgPulse(0, SyncPulseProg)


for i, ex_pulse_length in enumerate(pulse_lengths):
   
    #Возбуждающий импульс переменной длины
    exPulseI = RectSine(settings_ex[awg_ex_I_ch]['A'], 1./ex_if, ex_pulse_length, 
                        settings_ex[awg_ex_I_ch]['P'], 
                        settings_ex[awg_ex_I_ch]['O'], max_pulse_length - ex_pulse_length, sample_period = 1./SampleRate_ex)
    exPulseQ = RectSine(settings_ex[awg_ex_Q_ch]['A'], 1./ex_if,
                        ex_pulse_length, settings_ex[awg_ex_Q_ch]['P'], settings_ex[awg_ex_Q_ch]['O'],
                        max_pulse_length - ex_pulse_length, sample_period = 1./SampleRate_ex)

    awg_tek.run()
    exPulseI[0] = settings_ex[awg_ex_I_ch]['O']
    exPulseQ[0] = settings_ex[awg_ex_Q_ch]['O']
    #Программируем импульсы в awg
    awg_ex.LoadRAM(awg_ex_I_ch, exPulseI[0:data_len_ex])
    awg_ex.LoadRAM(awg_ex_Q_ch, exPulseQ[0:data_len_ex])
    
    for software_average_id in range(software_averages):
        
        #Мерием по триггеру
        daq0.start_with_trigger_and_waitready()
        #Усредняем результат
        measI[i] += np.sum(daq0.readout_doublechannel_multimode_bin()[0],
                           axis=0)
        measQ[i] += np.sum(daq0.readout_doublechannel_multimode_bin()[1],
                           axis=0)
        measI[i] -=np.mean(measI[i]) 
        measQ[i] -=np.mean(measQ[i])
        
        daq0.stop()
     
    #Делаем преобразованеи до нулевой частоты 
    I_Rabi[i] = np.mean(measI[i]*dm_cos - measQ[i]*dm_sin)/(nums*software_averages)
    Q_Rabi[i] = np.mean(measQ[i]*dm_cos + measI[i]*dm_sin)/(nums*software_averages)
      
    #Рисуем графики
    plt.figure("I_Rabi")
    amp_plot.set_data(pulse_lengths[:i+1], I_Rabi[:i+1])
    plt.ylim((np.min(I_Rabi[:i+1]), np.max(I_Rabi[:i+1])))
    plt.figure("Q_Rabi")
    ph_plot.set_data(pulse_lengths[:i+1], Q_Rabi[:i+1])
    plt.ylim((np.min(Q_Rabi[:i+1]), np.max(Q_Rabi[:i+1])))
    plt.pause(0.01)   

4096


C:\Python27\Lib\site-packages\matplotlib\axes\_base.py:2809: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=0.263702185712, top=0.263702185712
  'bottom=%s, top=%s') % (bottom, top))
C:\Python27\Lib\site-packages\matplotlib\axes\_base.py:2809: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=-0.0385615930668, top=-0.0385615930668
  'bottom=%s, top=%s') % (bottom, top))


In [17]:
plt.plot(pulse_lengths[:i+1], Q_Rabi[:i+1])

C:\Python27\Lib\site-packages\matplotlib\collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [18]:
plt.pcolormesh(measI)

# Rabi with Gaussian-convolved rect pulses

In [14]:
#Длительности возбуждающих импульсов
pulse_lengths = np.arange(0., 10e-6, 4e-9)

PulsePeriod = 20e-6

ro_pulse_length = 1e-6
#Длительность выборки DAQ
SampleRate = 500e6
nums = 1000
Range = 50

software_averages = 10

lo1.set_frequency(ex_lo_freq)
pna.set_cw_freq(ro_lo_freq)

#Настраиваем awg
data_len = int(PulsePeriod/2e-9)+1 
awg.set_repetition_period(PulsePeriod)

#Инициализация DAQ
signal_size = int(np.ceil(ro_pulse_length*3.*SampleRate))
segsize = 2*int( 2**np.ceil(np.log2(signal_size)) )
print segsize
posttrigger = segsize*3/4
signal_start = segsize - posttrigger 
signal_end = segsize - posttrigger + signal_size + zero_delay*2
if signal_end>segsize:
    signal_end = segsize
segsize = 2048
posttrigger = 512*3
    
daq0.stop()
daq0.set_spc_samplerate(SampleRate)
daq0.init_channel01_multiple_recording(rate=SampleRate, nums=nums, segsize=segsize, 
                                  posttrigger=posttrigger, amp0=Range, amp1=Range)
daq0.trigger_termination_50Ohm()
daq0.buffer_setup()


lo1.set_status(1)
#segsize = signal_end -  signal_start
#Контейнеры для данных
measQ = np.zeros((len(pulse_lengths), segsize))
measI = np.zeros((len(pulse_lengths), segsize))
I_Rabi = np.zeros( len(pulse_lengths) )
Q_Rabi = np.zeros( len(pulse_lengths) )

#Функции для демодуляции
t = linspace(0.,segsize/SampleRate, segsize, endpoint=False)
dm_sin = sin(2.*pi*ro_if*t)
dm_cos = cos(2.*pi*ro_if*t)

#Графики
plt.figure("I_Rabi")
amp_plot = plt.plot(pulse_lengths, I_Rabi)[0]
plt.figure("Q_Rabi")
ph_plot = plt.plot(pulse_lengths, Q_Rabi)[0]

max_pulse_length = np.max(pulse_lengths)

#Считывающий импульс сразу после возбуждающего
roPulseI = RectSine(settings_ro[awg_ro_I_ch]['A'], 1./ro_if, ro_pulse_length,
                    settings_ro[awg_ro_I_ch]['P'], settings_ro[awg_ro_I_ch]['O'], max_pulse_length+500e-9)
roPulseQ = RectSine(settings_ro[awg_ro_Q_ch]['A'], 1./ro_if, ro_pulse_length, 
                    settings_ro[awg_ro_Q_ch]['P'], settings_ro[awg_ro_Q_ch]['O'], max_pulse_length+500e-9)
awg_ro.LoadRAM(awg_ro_I_ch, roPulseI)
awg_ro.LoadRAM(awg_ro_Q_ch, roPulseQ)
awg_ex.set_trigger_mode('TRIG')
# setting trigger to start readout immideately after excitation puls
# zero_delay надо сначала померить в delay synchronization
#triggerPulse = [0, zero_delay+int(max_pulse_length/1e-9), 3, 100, 3, 1, 0, 4]
#SyncPulseProg = awg.intToBytes(triggerPulse)
#awg.ProgPulse(0, SyncPulseProg)


for i, ex_pulse_length in enumerate(pulse_lengths):
   
    #Возбуждающий импульс переменной длины
    exPulseI = RectSine(settings_ex[awg_ex_I_ch]['A'], 1./ex_if, ex_pulse_length, 
                        settings_ex[awg_ex_I_ch]['P'], 
                        settings_ex[awg_ex_I_ch]['O'], max_pulse_length - ex_pulse_length)
    exPulseQ = RectSine(settings_ex[awg_ex_Q_ch]['A'], 1./ex_if,
                        ex_pulse_length, settings_ex[awg_ex_Q_ch]['P'], settings_ex[awg_ex_Q_ch]['O'],
                        max_pulse_length - ex_pulse_length)

    awg_tek.run()
    exPulseI[0] = settings_ex[awg_ex_I_ch]['O']
    exPulseQ[0] = settings_ex[awg_ex_Q_ch]['O']
    #Программируем импульсы в awg
    awg_ex.LoadRAM(awg_ex_I_ch, exPulseI[0:data_len])
    awg_ex.LoadRAM(awg_ex_Q_ch, exPulseQ[0:data_len])
    
    for software_average_id in range(software_averages):
        
        #Мерием по триггеру
        daq0.start_with_trigger_and_waitready()
        #Усредняем результат
        measI[i] += np.sum(daq0.readout_doublechannel_multimode_bin()[0],
                           axis=0)
        measQ[i] += np.sum(daq0.readout_doublechannel_multimode_bin()[1],
                           axis=0)
        measI[i] -=np.mean(measI[i]) 
        measQ[i] -=np.mean(measQ[i])
        
        daq0.stop()
     
    #Делаем преобразованеи до нулевой частоты 
    I_Rabi[i] = np.mean(measI[i]*dm_cos - measQ[i]*dm_sin)/(nums*software_averages)
    Q_Rabi[i] = np.mean(measQ[i]*dm_cos + measI[i]*dm_sin)/(nums*software_averages)
      
    #Рисуем графики
    plt.figure("I_Rabi")
    amp_plot.set_data(pulse_lengths[:i+1], I_Rabi[:i+1])
    plt.ylim((np.min(I_Rabi[:i+1]), np.max(I_Rabi[:i+1])))
    plt.figure("Q_Rabi")
    ph_plot.set_data(pulse_lengths[:i+1], Q_Rabi[:i+1])
    plt.ylim((np.min(Q_Rabi[:i+1]), np.max(Q_Rabi[:i+1])))
    plt.pause(0.01)   

4096


C:\Python27\Lib\site-packages\matplotlib\axes\_base.py:2809: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=0.000657342342055, top=0.000657342342055
  'bottom=%s, top=%s') % (bottom, top))
C:\Python27\Lib\site-packages\matplotlib\axes\_base.py:2809: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=-1.78596610408e-05, top=-1.78596610408e-05
  'bottom=%s, top=%s') % (bottom, top))


In [27]:
I_Rabi = np.mean((np.asarray(measI)*dm_cos - np.asarray(measQ)*dm_sin)[:, 1041:], axis=1)
Q_Rabi = np.mean((np.asarray(measI)*dm_cos + np.asarray(measQ)*dm_sin)[:, 1041:], axis=1)
plt.figure()
plt.plot(I_Rabi)
plt.figure()
plt.plot(Q_Rabi)

In [29]:
plt.plot(np.abs(S21))

In [31]:
# фитует результаты измерений экспонентой
S21 = I_Rabi+1j*Q_Rabi
f = lambda x,A0r,Aoi,A1r,A1i,t,fR,phi0,eq_pop: (A0r+A0i*1j+(A1r+A1i*1j)*(eq_pop+np.exp(-x/t)*(np.cos(2*np.pi*fR*x+phi0)-eq_pop)))
cost = lambda p: np.abs(f(pulse_lengths,*p)-S21)**2
# изначальные параметры:
Rabi_fft=np.fft.fft(S21-np.mean(S21))
frequencies = np.fft.fftfreq(len(pulse_lengths), pulse_lengths[1]-pulse_lengths[0])
plt.plot(frequencies, np.abs(Rabi_fft))
fR = frequencies[np.argmax(np.abs(Rabi_fft))]
A0r =np.mean(S21.real)
A0i =np.mean(S21.imag)
A1r = np.sqrt(2)*np.std(S21.real)
A1i = np.sqrt(2)*np.std(S21.imag)
phi0 = 0
t = np.mean(pulse_lengths)
eq_pop = 0.5
from scipy.optimize import leastsq
fitresults = leastsq(cost, (A0r,A0i,A1r,A1i,t,fR,phi0,eq_pop))
fitted_curve = (pulse_lengths, np.log10(np.abs(f(pulse_lengths, *(fitresults[0]))))*10, np.unwrap(np.angle(f(pulse_lengths, *(fitresults[0])))))

In [21]:
# фитует результаты измерений

f = lambda x, A0, A1, t, fR, phi: A0 + A1*np.exp(-x/t)*(np.cos(2*np.pi*fR*x + phi))
#########################################
fit_res={'I':zeros(3),'Q':zeros(3), 'Ierr': zeros(3), 'Qerr': zeros(3)}

for dat,id in zip([I_Rabi, Q_Rabi],['I','Q']):
    fR = 10e6
    A0 =np.mean(dat)
    A1 = np.sqrt(2)*np.std(dat)
    phi = 0.
    t = 0.4e-6
    (res, err) = so.curve_fit(f, pulse_lengths, dat, p0 = (A0, A1, t, fR, phi))
    fit_res[id] = res
    fit_res[id+'err'] = np.sqrt(np.diag(err))
    plt.figure(id)
    plt.plot(pulse_lengths,  dat, 'o')
    plt.plot(pulse_lengths,  f(pulse_lengths, *tuple(res) ),
         label = "T_Rabi={:.2f}us\nF_Rabi={:.2f}MHz".format(res[2]*1e6, res[3]*1e-6), lw = 2, color = 'red')
    plt.legend()
print fit_res 

{'I': array([  1.98760343e-01,   6.08047562e-02,   4.89290358e-09,
         3.83571898e+04,  -3.31902629e-02]), 'Q': array([  1.88660844e-01,  -2.27218252e-01,   1.33342517e-10,
         1.01908594e+07,   2.48466948e-04]), 'Qerr': array([ inf,  inf,  inf,  inf,  inf]), 'Ierr': array([  7.28098784e-04,   3.06877112e+02,   1.64342324e-06,
         1.17465832e+10,   1.93463255e+05])}


C:\Python27\Lib\site-packages\IPython\kernel\__main__.py:3: RuntimeWarning: overflow encountered in exp
  app.launch_new_instance()


In [30]:
#Save with plot
header = {'date':datetime.datetime.now(), 'type': 'Rabi', 'name': 'Fliegenklatsche', 
          'ro_pulse_length':ro_pulse_length,  'Rabi frequency, Hz':np.abs(fitresults[0][5]), 'Rabi oscillation decay time, s': np.abs(fitresults[0][4])}
annotation = 'Rabi frequency: ${0:2.2f} \mathrm{{MHz}}$, Decay time: ${1:2.2f} \mu\mathrm{{s}}$\nExcitation freq: {2:1.4} GHz'.format(\
                        np.abs(fitresults[0][5])/1e6, fitresults[0][4]*1e6, ex_freq/1e9)

measurement =        ((pulse_lengths, np.asarray([0]), 
                       np.log10(np.abs(S21))*10, np.unwrap(np.angle(S21))))

save_pkl(header, measurement, curve_fit=fitted_curve, annotation=annotation)

In [173]:
#Save 
filename = 'rabi_1'
save_pkl_noplt(filename, {"I": I_Rabi , "Q": Q_Rabi, "fit_res": fit_res, "pulse_lenghts":pulse_lengths,
                "fit_func": "lambda x, A0, A1, t, fR, phi: A0 + A1*np.exp(-x/t)*(np.cos(2*np.pi*fR*x + phi))",
                "ex_freq":ex_freq,"ro_freq": ro_freq} )


In [167]:
1./fit_res['I'][3]/2.

1.5603623306426543e-07

In [43]:
#Load
pkl = load_pkl('rabi5')
Amp = pkl["Amp"]
Ph = pkl["Ph"]
pulse_lengths = pkl["pulse_lenghts"]

# Single-shot readout classifier data acquisition

In [17]:
hardware_samplerate = 500e6
nums = 5000
ro_pulse_length = 2e-6
ro_readout_length = 4e-6
hardware_segsize = int(2**np.ceil(np.log(ro_readout_length*hardware_samplerate)/np.log(2))*2)
posttrigger = 3*hardware_segsize/4
amp = 50
offs = 0
timeout = 20000
PulsePeriod = 40e-6
reprate = 1/PulsePeriod
software_averages = 1
software_binning = 1
binned_segsize = hardware_segsize/software_binning
center_freq_point = int(settings_ro[awg_ro_I_ch]['F']/hardware_samplerate*hardware_segsize)

awg_ex.set_repetition_period(PulsePeriod-1e-6)
awg_ro.set_repetition_period(PulsePeriod)

lo1.set_status(1)
lo1.set_frequency(ex_freq-settings_ex[awg_ex_I_ch]['F'])
pna.set_cw_freq(ro_freq-settings_ro[awg_ro_I_ch]['F'])
pulse = 'on'
if pulse == 'on':
    pulse_length = 1/(2*17e6)
else:
    pulse_length = 0
samplesI = np.zeros((binned_segsize, software_averages*nums), dtype=np.int8)
samplesQ = np.zeros((binned_segsize, software_averages*nums), dtype=np.int8)

exPulseI = RectSine(settings_ex[awg_ex_I_ch]['A'], 1./settings_ex[awg_ex_I_ch]['F'], pulse_length, 0, 
                            settings_ex[awg_ex_I_ch]['O'], 0, sample_period = 1./SampleRate_ex)
exPulseQ = RectSine(settings_ex[awg_ex_Q_ch]['A'], 1./settings_ex[awg_ex_Q_ch]['F'], pulse_length, 
                            settings_ex[awg_ex_Q_ch]['P'], settings_ex[awg_ex_Q_ch]['O'], 0, sample_period = 1./SampleRate_ex)
roPulseI = RectSine(settings_ro[awg_ro_I_ch]['A'], 1./settings_ro[awg_ro_I_ch]['F'], ro_pulse_length, 0, 
                        settings_ro[awg_ro_I_ch]['O'], pulse_length+500e-9, sample_period = 1./SampleRate_ro)
roPulseQ = RectSine(settings_ro[awg_ro_Q_ch]['A'], 1./settings_ro[awg_ro_Q_ch]['F'], ro_pulse_length, 
                        settings_ro[awg_ro_Q_ch]['P'], settings_ro[awg_ro_Q_ch]['O'], pulse_length+500e-9, sample_period = 1./SampleRate_ro)
# setting trigger to start readout after excitation pulses
#triggerPulse = [0, zero_delay+int(pulse_length/1e-9), 3, 100, 3, 1, 0, 4]
#SyncPulseProg = awg.intToBytes(triggerPulse)
#awg.ProgPulse(0, SyncPulseProg)


roPulseI[0] = settings_ro[awg_ro_I_ch]['O']
roPulseQ[0] = settings_ro[awg_ro_Q_ch]['O']
exPulseI[0] = settings_ex[awg_ex_I_ch]['O']
exPulseQ[0] = settings_ex[awg_ex_Q_ch]['O']

awg_ex.LoadRAM(awg_ex_I_ch, exPulseI)
awg_ex.LoadRAM(awg_ex_Q_ch, exPulseQ)
awg_ro.LoadRAM(awg_ro_I_ch, roPulseI)
awg_ro.LoadRAM(awg_ro_Q_ch, roPulseQ)
    
#plt.plot(exPulseI)
#plt.plot(roPulseQ)
    
daq0.stop()
daq0.init_channel01_multiple_recording(rate=SampleRate, nums=nums, segsize=hardware_segsize, 
                                  posttrigger=posttrigger, amp0=amp, amp1=amp)
daq0.buffer_setup()
for software_average_id in range(software_averages):
    daq0.trigger_termination_50Ohm()
    daq0.start_with_trigger_and_waitready()
    data = np.sum(np.reshape(daq0.readout_doublechannel_multimode_bin()[0], (nums, binned_segsize, software_binning)), axis=2).T
    samplesI[:,software_average_id*nums:(software_average_id+1)*nums] = data
    data = np.sum(np.reshape(daq0.readout_doublechannel_multimode_bin()[1], (nums, binned_segsize, software_binning)), axis=2).T
    samplesQ[:,software_average_id*nums:(software_average_id+1)*nums] = data
    daq0.stop()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<ipython-input-17-f2900ce6be99> in <module>()
     34                             settings_ex[awg_ex_Q_ch]['P'], settings_ex[awg_ex_Q_ch]['O'], 0, sample_period = 1./SampleRate_ex)
     35 roPulseI = RectSine(settings_ro[awg_ro_I_ch]['A'], 1./settings_ro[awg_ro_I_ch]['F'], ro_pulse_length, 0, 
---> 36                         settings_ro[awg_ro_I_ch]['O'], pulse_length+500e-9, sample_period = 1./SampleRate_ro)
     37 roPulseQ = RectSine(settings_ro[awg_ro_Q_ch]['A'], 1./settings_ro[awg_ro_Q_ch]['F'], ro_pulse_length, 
     38                         settings_ro[awg_ro_Q_ch]['P'], settings_ro[awg_ro_Q_ch]['O'], pulse_length+500e-9, sample_period = 1./SampleRate_ro)

NameError: name 'SampleRate_ro' is not defined


In [20]:
plt.figure("samples4")
plt.plot(np.mean(samplesI, axis=1))
plt.plot(np.mean(samplesQ, axis=1))

limit_min = 1310
limit_max = 2450

t = linspace(0.,hardware_segsize/SampleRate, hardware_segsize, endpoint=False)
dm_sin = sin(2.*pi*ro_if*t)
dm_cos = cos(2.*pi*ro_if*t)
I = np.mean((samplesI.T*dm_cos - samplesQ.T*dm_sin)[:,limit_min:limit_max], axis=1)/(nums*software_averages)
Q = np.mean((samplesQ.T*dm_cos + samplesI.T*dm_sin)[:,limit_min:limit_max], axis=1)/(nums*software_averages)
plt.figure("single-shot3")
if pulse == 'on':
    color1 = 'green'
    color2 = 'black'
else:
    color1 = 'red'
    color2 = 'blue'
plt.scatter(I,Q, color=color1, marker='>',s=1)
plt.scatter(np.mean(I), np.mean(Q), color=color2, marker='o',s=30)
plt.xlim([np.min(I), np.max(I)])
plt.ylim([np.min(Q), np.max(Q)])
#plt.axis('equal')

(-0.00016741688807382282, 0.00024374791131208514)

In [18]:
print ('I', pulse, ':', np.mean(I), np.std(I))
print ('Q', pulse, ':', np.mean(Q), np.std(Q))

('I', 'off', ':', -4.1202480949819422e-05, 5.6515861695848651e-05)
('Q', 'off', ':', -3.1147263176621912e-05, 4.6948002425219549e-05)


In [21]:
print ('I', pulse, ':', np.mean(I), np.std(I))
print ('Q', pulse, ':', np.mean(Q), np.std(Q))

('I', 'on', ':', -5.3737044701352243e-05, 4.7444042073860219e-05)
('Q', 'on', ':', 2.8335766494632365e-05, 5.7062475438355504e-05)


In [16]:
import pickle
pickle.dump(samples, open('C:/qtlab/ReiData/data/Nov 13 2016/Single-shot readout samples excite length 0.pkl', 'wb'))

# Rabi 2D

In [19]:
hardware_samplerate = 500e6
nums = 1000
ro_pulse_length = 2e-6
ro_readout_length = 3e-6
hardware_segsize = int(2**np.ceil(np.log(ro_readout_length*hardware_samplerate)/np.log(2))*2)
amp = 50
offs = 0
timeout = 20000
PulsePeriod= 20e-6
reprate = 1/PulsePeriod
software_averages = 4
#software_averages = 500
software_binning = 1
binned_segsize = hardware_segsize/software_binning
#center_freq_point = int(settings_ro[awg_ro_I_channel]['F']/hardware_samplerate*hardware_segsize)

ex_freq_old = ex_freq
ex_freqs = ex_freq + np.linspace(-18.5e6, 18.5e6, 71)
pulse_lengths = np.linspace(0, 0.7e-6, 71)
max_pulse_length = np.max(pulse_lengths)

hardware_segsize = int(2**np.ceil(np.log((ro_readout_length+max_pulse_length)*hardware_samplerate)/np.log(2))*2)
posttrigger = 3*hardware_segsize/4

awg_ex.set_repetition_period(PulsePeriod-1e-6)
awg_ro.set_repetition_period(PulsePeriod)

lo1.set_status(1)
pna.set_cw_freq(ro_freq-settings_ro[awg_ro_I_ch]['F'])
#sample = np.zeros((binned_segsize, len(pulse_lengths), len(excite_freqs)))
#covariation_matrix = np.zeros((binned_segsize, binned_segsize, len(pulse_lengths), len(excite_freqs)))
#S21 = np.zeros((len(pulse_lengths), len(excite_freqs)), dtype=np.complex)
#ro_results_output = open('C:/qtlab/ReiData/data/Feb 19 2016/rabi_dump/dump1.pkl', 'w')

#Функции для демодуляции
signal_start = int(max_pulse_length*SampleRate+(hardware_segsize-posttrigger))
signal_end = signal_start+int(ro_readout_length*SampleRate)
signal_length = signal_end-signal_start

t = linspace(0.,signal_length/SampleRate, signal_length, endpoint=False)
dm_sin = sin(2.*pi*ro_if*t)
dm_cos = cos(2.*pi*ro_if*t)

measQ = np.zeros((len(pulse_lengths), len(ex_freqs), signal_length))
measI = np.zeros((len(pulse_lengths), len(ex_freqs), signal_length))
I_Rabi = np.zeros( ( len(pulse_lengths), len(ex_freqs) ))
Q_Rabi = np.zeros( (len(pulse_lengths), len(ex_freqs) ))

daq0.stop()
daq0.set_spc_samplerate(SampleRate)
daq0.init_channel01_multiple_recording(rate=SampleRate, nums=nums, segsize=hardware_segsize, 
                                  posttrigger=posttrigger, amp0=amp, amp1=amp)
daq0.trigger_termination_50Ohm()
daq0.buffer_setup()


try:
    for j, ex_freq in enumerate(ex_freqs):
        lo1.set_frequency(ex_freq-settings_ex[awg_ex_I_ch]['F'])
        #Считывающий импульс сразу после возбуждающего
        roPulseI = RectSine(settings_ro[awg_ro_I_ch]['A'], 1./ro_if, ro_pulse_length,
                    settings_ro[awg_ro_I_ch]['P'], settings_ro[awg_ro_I_ch]['O'], max_pulse_length+500e-9, sample_period = 1./SampleRate_ro)
        roPulseQ = RectSine(settings_ro[awg_ro_Q_ch]['A'], 1./ro_if, ro_pulse_length, 
                    settings_ro[awg_ro_Q_ch]['P'], settings_ro[awg_ro_Q_ch]['O'], max_pulse_length+500e-9, sample_period = 1./SampleRate_ro)
        roPulseI[0] = settings_ro[awg_ro_I_ch]['O']
        roPulseQ[0] = settings_ro[awg_ro_Q_ch]['O']

        awg_ro.LoadRAM(awg_ro_I_ch, roPulseI)
        awg_ro.LoadRAM(awg_ro_Q_ch, roPulseQ)

        for i, ex_pulse_length in enumerate(pulse_lengths):
            #Возбуждающий импульс переменной длины
            excite_pulse_length = ex_pulse_length
            exPulseI = RectSine(settings_ex[awg_ex_I_ch]['A'], 1./ex_if, ex_pulse_length, 
                        settings_ex[awg_ex_I_ch]['P'], 
                        settings_ex[awg_ex_I_ch]['O'], max_pulse_length - ex_pulse_length, sample_period = 1./SampleRate_ex)
            exPulseQ = RectSine(settings_ex[awg_ex_Q_ch]['A'], 1./ex_if,
                        ex_pulse_length, settings_ex[awg_ex_Q_ch]['P'], settings_ex[awg_ex_Q_ch]['O'],
                        max_pulse_length - ex_pulse_length, sample_period = 1./SampleRate_ex)
            
            exPulseI[0] = settings_ex[awg_ex_I_ch]['O']
            exPulseQ[0] = settings_ex[awg_ex_Q_ch]['O']
    
            #Программируем импульсы в awg
            awg_ex.LoadRAM(awg_ex_I_ch, exPulseI)
            awg_ex.LoadRAM(awg_ex_Q_ch, exPulseQ)

            for software_average_id in range(software_averages):
                #Мерием по триггеру
                daq0.start_with_trigger_and_waitready()
                #Усредняем результат
                measI[i,j,:] += np.sum(daq0.readout_doublechannel_multimode_bin()[0],
                           axis=0)[signal_start:signal_end]
                measQ[i,j,:] += np.sum(daq0.readout_doublechannel_multimode_bin()[1],
                            axis=0)[signal_start:signal_end]
                #measI[i,j,:] -=np.mean(measI[i,j]) 
                #measQ[i,j,:] -=np.mean(measQ[i,j])
        
                daq0.stop()
     
            #Делаем преобразованеи до нулевой частоты 
            I_Rabi[i,j] = np.mean(measI[i,j]*dm_cos - measQ[i,j]*dm_sin)/(nums*software_averages)
            Q_Rabi[i,j] = np.mean(measQ[i,j]*dm_cos + measI[i,j]*dm_sin)/(nums*software_averages)
      
        plt.figure(1)
        plt.clf()
        plt.pcolor(ex_freqs[:j+1], pulse_lengths, I_Rabi[:,:j+1])
        plt.colorbar()
        plt.figure(2)
        plt.clf()
        plt.pcolor(ex_freqs[:j+1], pulse_lengths, Q_Rabi[:,:j+1])
        plt.colorbar()
        plt.pause(0.1)
        last_pulse_length_point = i
        last_excite_freq_point = j
#ro_results_output.close()
finally:
    ex_freq = ex_freq_old

In [52]:
plt.figure()
plt.pcolormesh(measI[:,:])

In [12]:
(ex_freqs[:j+1], pulse_lengths, measI[:,:j+1])

(array([  6.61030000e+09]),
 array([  0.00000000e+00,   1.00000000e-08,   2.00000000e-08,
          3.00000000e-08,   4.00000000e-08,   5.00000000e-08,
          6.00000000e-08,   7.00000000e-08,   8.00000000e-08,
          9.00000000e-08,   1.00000000e-07,   1.10000000e-07,
          1.20000000e-07,   1.30000000e-07,   1.40000000e-07,
          1.50000000e-07,   1.60000000e-07,   1.70000000e-07,
          1.80000000e-07,   1.90000000e-07,   2.00000000e-07]),
 array([[[  191.,   488.,   383., ...,    70.,   174.,   581.]],
 
        [[ 1132.,   958.,  1220., ...,   905.,  1048.,   847.]],
 
        [[  835.,   797.,  1164., ...,  1110.,  1023.,   889.]],
 
        ..., 
        [[ 1306.,  1269.,  1296., ...,   809.,   976.,   718.]],
 
        [[  897.,  1215.,  1102., ...,  1286.,  1525.,  1327.]],
 
        [[  815.,  1584.,  1338., ...,   832.,  1443.,   861.]]]))

In [20]:
#Save
header = {'date':datetime.datetime.now(), 'type': 'Rabi2D', 'name': 'Fliegenklatsche'}
save_pkl(header, ex_freqs, pulse_lengths, I_Rabi, Q_Rabi)

In [25]:
S21 = I_Rabi+1j*Q_Rabi
sample = measI + 1j* measQ
execfile('save_pkl.py')
time_domain_box_setup = {'use_coupler':True, 'excite_lo':'lo1', 'ro_lo':'pna', 
                         'excite_lo_freq':ex_lo_freq, 'ro_lo_freq':ro_lo_freq, }
#dac_setup = {'samplerate': samplerate, 'zero_delay': zero_delay, 'sample_size': binned_segsize, 'posttrigger': posttrigger,
#          'amp': amp, 'offs': offs,
#          'hardware_averages': nums, 'software_averages': software_averages,}
header = {'date':datetime.datetime.now(), 'type': 'Rabi 2D', 'name': '', 'pulse_settings': settings, 
          'ro_pulse_length':ro_pulse_length}#, 'excite_pulse_length':ex_pulse_length,
          #'dac_setup':dac_setup, 
          #'time_domain_box_setup':time_domain_box_setup}
sample_axis = np.linspace((posttrigger-hardware_segsize)/hardware_samplerate, posttrigger/hardware_samplerate, binned_segsize)
measurement =        ((pulse_lengths, ex_freqs[:last_excite_freq_point+1],
                       np.log10(np.abs(S21[:,:last_excite_freq_point+1]))*10, np.angle(S21[:,:last_excite_freq_point+1])))
sample_measurement = ((np.linspace((posttrigger-hardware_segsize)/hardware_samplerate, posttrigger/hardware_samplerate, binned_segsize), 
                       pulse_lengths, ex_freqs[:last_excite_freq_point+1],
                       sample[:,:,:last_excite_freq_point+1], np.zeros(sample[:,:,:last_excite_freq_point+1].shape)))
#covariation_matrix_measurement = ((np.linspace((posttrigger-hardware_segsize)/hardware_samplerate, posttrigger/hardware_samplerate, binned_segsize), 
#                       np.linspace((posttrigger-hardware_segsize)/hardware_samplerate, posttrigger/hardware_samplerate, binned_segsize), 
#                       pulse_lengths, excite_freqs[:last_excite_freq_point+1],
#                       covariation_matrix[:,:,:,:last_excite_freq_point+1], np.zeros(covariation_matrix[:,:,:,:last_excite_freq_point+1].shape)))
#extended_covariation_matrix = np.zeros((binned_segsize+1, binned_segsize+1, len(pulse_lengths), last_excite_freq_point+1))
#extended_covariation_matrix[0,0,:,:] = software_averages*nums
#extended_covariation_matrix[1:,0,:,:] = sample[:,:,:last_excite_freq_point+1]
#extended_covariation_matrix[0,1:,:,:] = sample[:,:,:last_excite_freq_point+1]
#extended_covariation_matrix[1:,1:,:,:] = covariation_matrix[:,:,:,:last_excite_freq_point+1]
#extended_sample_axis = np.append(np.asarray([np.nan]), sample_axis)
header['type'] = 'Rabi 2D S21'
save_pkl(header, measurement)
header['type'] = 'Rabi 2D oscillogram'
save_pkl(header, sample_measurement)
#header['type'] = 'Rabi 2D covariation matrix'
#save_pkl(header, covariation_matrix_measurement)
#header['type'] = 'Rabi 2D extended covariation matrix'
#save_pkl(header, (extended_sample_axis, extended_sample_axis, pulse_lengths, excite_freqs[:last_excite_freq_point+1], 
#                  extended_covariation_matrix, np.zeros(extended_covariation_matrix.shape)))

# Ramsey fringes

In [77]:
#Длительности возбуждающих импульсов
pulse_lengths = np.arange(0., 4e-6, 12e-9)
pulse_length_pi2 = 5e-9

PulsePeriod = 20e-6

ro_pulse_length = 1e-6
#Длительность выборки DAQ
SampleRate = 500e6
nums = 1000
Range = 50
SampleRate_ro = SampleRate
SampleRate_ex = SampleRate*2
software_averages = 10
freq_shift = 12e6

lo1.set_frequency(ex_lo_freq)
pna.set_cw_freq(ro_lo_freq)

awg.set_repetition_period(PulsePeriod)
awg_tek.set_repetition_period(PulsePeriod-1e-6)

#Настраиваем awg
data_len_ex = int(PulsePeriod*SampleRate_ex)*2+1 
data_len_ro = int(PulsePeriod*SampleRate_ro)+1 
awg.set_repetition_period(PulsePeriod)
awg_ex.set_trigger_mode('TRIG')

#Инициализация DAQ
signal_size = int(np.ceil((np.max(pulse_lengths)+ro_pulse_length)*SampleRate_ro))
segsize = 2*int( 2**np.ceil(np.log2(signal_size)) )
print segsize
posttrigger = segsize*3/4
signal_start = segsize - posttrigger 
signal_end = segsize - posttrigger + signal_size + zero_delay*2
if signal_end>segsize:
    signal_end = segsize
#segsize = 2048
#posttrigger = 512*3
    
daq0.stop()
daq0.set_spc_samplerate(SampleRate)
daq0.init_channel01_multiple_recording(rate=SampleRate, nums=nums, segsize=segsize, 
                                  posttrigger=posttrigger, amp0=Range, amp1=Range)
daq0.trigger_termination_50Ohm()
daq0.buffer_setup()


lo1.set_status(1)
#segsize = signal_end -  signal_start
#Контейнеры для данных
measQ = np.zeros((len(pulse_lengths), segsize))
measI = np.zeros((len(pulse_lengths), segsize))
I_Rabi = np.zeros( len(pulse_lengths) )
Q_Rabi = np.zeros( len(pulse_lengths) )

#Функции для демодуляции
t = linspace(0.,segsize/SampleRate, segsize, endpoint=False)
dm_sin = sin(2.*pi*ro_if*t)
dm_cos = cos(2.*pi*ro_if*t)

#Графики
plt.figure("I_Rabi")
amp_plot = plt.plot(pulse_lengths, I_Rabi)[0]
plt.figure("Q_Rabi")
ph_plot = plt.plot(pulse_lengths, Q_Rabi)[0]

max_pulse_length = np.max(pulse_lengths)+2*pulse_length_pi2

#Считывающий импульс сразу после возбуждающего
roPulseI = RectSine(settings_ro[awg_ro_I_ch]['A'], 1./ro_if, ro_pulse_length,
                    settings_ro[awg_ro_I_ch]['P'], settings_ro[awg_ro_I_ch]['O'], max_pulse_length+500e-9, sample_period = 1./SampleRate_ro)
roPulseQ = RectSine(settings_ro[awg_ro_Q_ch]['A'], 1./ro_if, ro_pulse_length, 
                    settings_ro[awg_ro_Q_ch]['P'], settings_ro[awg_ro_Q_ch]['O'], max_pulse_length+500e-9, sample_period = 1./SampleRate_ro)
awg_ro.LoadRAM(awg_ro_I_ch, roPulseI)
awg_ro.LoadRAM(awg_ro_Q_ch, roPulseQ)
# setting trigger to start readout immideately after excitation puls
# zero_delay надо сначала померить в delay synchronization
#triggerPulse = [0, zero_delay+int(max_pulse_length/1e-9), 3, 100, 3, 1, 0, 4]
#SyncPulseProg = awg.intToBytes(triggerPulse)
#awg.ProgPulse(0, SyncPulseProg)


for i, ex_pulse_length in enumerate(pulse_lengths):
   
    #Возбуждающий импульс переменной длины
    exPulseI = RectSine(settings_ex[awg_ex_I_ch]['A'], 1./ex_if, pulse_length_pi2, 
                        settings_ex[awg_ex_I_ch]['P'], 
                        settings_ex[awg_ex_I_ch]['O']/2., max_pulse_length - ex_pulse_length - pulse_length_pi2*2, \
                        sample_period = 1./SampleRate_ex)
    exPulseQ = RectSine(settings_ex[awg_ex_Q_ch]['A'], 1./ex_if,
                        pulse_length_pi2, settings_ex[awg_ex_Q_ch]['P'], settings_ex[awg_ex_Q_ch]['O'],
                        max_pulse_length - ex_pulse_length - pulse_length_pi2*2, sample_period = 1./SampleRate_ex)

    exPulseI += RectSine(settings_ex[awg_ex_I_ch]['A'], 1./ex_if, pulse_length_pi2, 
                        settings_ex[awg_ex_I_ch]['P']+2*np.pi*(ex_if-freq_shift)*(ex_pulse_length + pulse_length_pi2), 
                        settings_ex[awg_ex_I_ch]['O']/2., max_pulse_length - pulse_length_pi2, sample_period = 1./SampleRate_ex)
    exPulseQ += RectSine(settings_ex[awg_ex_Q_ch]['A'], 1./ex_if,
                        pulse_length_pi2, settings_ex[awg_ex_Q_ch]['P']+2*np.pi*(ex_if-freq_shift)*(ex_pulse_length + pulse_length_pi2),\
                        settings_ex[awg_ex_Q_ch]['O'],
                        max_pulse_length - pulse_length_pi2, sample_period = 1./SampleRate_ex)
    
    awg_tek.run()
    exPulseI[0] = settings_ex[awg_ex_I_ch]['O']
    exPulseQ[0] = settings_ex[awg_ex_Q_ch]['O']
    #Программируем импульсы в awg
    awg_ex.LoadRAM(awg_ex_I_ch, exPulseI[0:data_len_ex])
    awg_ex.LoadRAM(awg_ex_Q_ch, exPulseQ[0:data_len_ex])
    
    for software_average_id in range(software_averages):
        
        #Мерием по триггеру
        daq0.start_with_trigger_and_waitready()
        #Усредняем результат
        measI[i] += np.sum(daq0.readout_doublechannel_multimode_bin()[0],
                           axis=0)
        measQ[i] += np.sum(daq0.readout_doublechannel_multimode_bin()[1],
                           axis=0)
        measI[i] -=np.mean(measI[i]) 
        measQ[i] -=np.mean(measQ[i])
        
        daq0.stop()
     
    #Делаем преобразованеи до нулевой частоты 
    I_Rabi[i] = np.mean(measI[i]*dm_cos - measQ[i]*dm_sin)/(nums*software_averages)
    Q_Rabi[i] = np.mean(measQ[i]*dm_cos + measI[i]*dm_sin)/(nums*software_averages)
      
    #Рисуем графики
    plt.figure("I_Rabi")
    amp_plot.set_data(pulse_lengths[:i+1], I_Rabi[:i+1])
    plt.ylim((np.min(I_Rabi[:i+1]), np.max(I_Rabi[:i+1])))
    plt.figure("Q_Rabi")
    ph_plot.set_data(pulse_lengths[:i+1], Q_Rabi[:i+1])
    plt.ylim((np.min(Q_Rabi[:i+1]), np.max(Q_Rabi[:i+1])))
    plt.pause(0.01)   

2048


C:\Python27\Lib\site-packages\matplotlib\axes\_base.py:2809: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=-0.0669658067686, top=-0.0669658067686
  'bottom=%s, top=%s') % (bottom, top))
C:\Python27\Lib\site-packages\matplotlib\axes\_base.py:2809: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=0.669171716715, top=0.669171716715
  'bottom=%s, top=%s') % (bottom, top))


In [48]:
plt.pcolormesh(measI)

In [78]:
S21 = I_Rabi+1j*Q_Rabi
# фитует результаты измерений экспонентой
delays = pulse_lengths
f = lambda x,A0r,Aoi,A1r,A1i,t,fR,phi0,eq_pop: (A0r+A0i*1j+(A1r+A1i*1j)*(eq_pop+np.exp(-x/t)*(np.cos(2*np.pi*fR*x+phi0)-eq_pop)))
cost = lambda p: np.abs(f(delays,*p)-S21)**2
# изначальные параметры:
Rabi_fft=np.fft.fft(S21-np.mean(S21))
frequencies = np.fft.fftfreq(len(delays), delays[1]-delays[0])
plt.plot(frequencies, np.abs(Rabi_fft))
fR = frequencies[np.argmax(np.abs(Rabi_fft))]
A0r =np.mean(S21.real)
A0i =np.mean(S21.imag)
A1r = np.sqrt(2)*np.std(S21.real)
A1i = np.sqrt(2)*np.std(S21.imag)
phi0 = 0
t = np.mean(delays)
eq_pop = 0.5
from scipy.optimize import leastsq
fitresults = leastsq(cost, (A0r,A0i,A1r,A1i,t,fR,phi0,eq_pop))
fitted_curve = (delays, np.log10(np.abs(f(delays, *(fitresults[0]))))*10, np.angle(f(delays, *(fitresults[0]))))
print (fitresults)

(array([ -1.27301762e-01,   5.56998453e-01,   6.19060103e-02,
         1.16375147e-01,   6.96382359e-07,  -1.76942232e+07,
        -1.74791737e-01,  -2.00824712e-01]), 5)


In [79]:
execfile('save_pkl.py')
time_domain_box_setup = {'use_coupler':True, 'excite_lo':'lo1', 'ro_lo':'pna', 
                         'excite_lo_freq':ex_lo_freq, 'ro_lo_freq':ro_lo_freq, }
dac_setup = {'samplerate': hardware_samplerate, 'zero_delay': zero_delay, 'sample_size': binned_segsize, 'posttrigger': posttrigger,
          'amp': 50, 'offs': 0,
          'hardware_averages': nums, 'software_averages': software_averages,}
header = {'date':datetime.datetime.now(), 'type': 'Ramsey', 'name': 'Fliegenklatsche Ramsey', 
          'ro_pulse_length':ro_pulse_length, 'excite_pulse_length':excite_pulse_length,
          'dac_setup':dac_setup, 'time_domain_box_setup':time_domain_box_setup, \
          'Ramsey oscillation frequency, Hz':np.abs(fitresults[0][5]), 'Ramsey oscillation decay time, s': np.abs(fitresults[0][4])}
annotation = 'Ramsey frequency: {0:2.2f} MHz, Decay time: {1:2.2} μs, \nExcitation freq: {2:1.5} GHz, offset freq: {3:2.2f} MHz'.format(\
            np.abs(fitresults[0][5])/1e6, fitresults[0][4]*1e6, ex_freq/1e9, frequency_offset/1e6)

measurement =        ((delays[:last_point+1], np.asarray([0]), 
                       np.log10(np.abs(S21[:last_point+1]))*10, np.angle(S21[:last_point+1])))

save_pkl(header, measurement, curve_fit=fitted_curve, annotation=annotation)

# Hahn Spin echo

In [9]:
hardware_samplerate = 500e6
nums = 5000
ro_pulse_length = 1e-6
ro_readout_length = 2e-6
hardware_segsize = int(2**np.ceil(np.log(ro_readout_length*hardware_samplerate)/np.log(2))*2)
posttrigger = 3*hardware_segsize/4
amp = 50
offs = 0
timeout = 20000
PulsePeriod = 40e-6
reprate = 1/PulsePeriod
software_averages = 20
software_binning = 1
binned_segsize = hardware_segsize/software_binning
center_freq_point = int(settings_ro[awg_ro_I_ch]['F']/hardware_samplerate*hardware_segsize)

lo1.set_status(1)
lo1.set_frequency(ex_freq-settings_ex[awg_ex_I_ch]['F'])
pna.set_cw_freq(ro_freq-settings_ro[awg_ro_I_ch]['F'])
frequency_offset = 0e6
excite_pulse_length = 724e-9/2
delays = np.linspace(0, 2.5e-6, 101)
max_delay_with_pulses = excite_pulse_length*4+np.max(delays)
sample = np.zeros((binned_segsize, len(delays)))
#covariation_matrix = np.zeros((binned_segsize, binned_segsize, len(delays)))
S21 = np.zeros((len(delays),), dtype=np.complex)
#ro_results_output = open('C:/qtlab/ReiData/data/Feb 19 2016/rabi_dump/dump1.pkl', 'w')

for delay_id, excite_delay in enumerate(delays):
    excitePulseI1 = RectSine(settings_ex[awg_ex_I_ch]['A'], 1./settings_ex[awg_ex_I_ch]['F'], excite_pulse_length, 0, 
                             settings_ex[awg_ex_I_ch]['O'], \
                             max_delay_with_pulses-excite_delay-excite_pulse_length*4, sample_period = 1./SampleRate_ex)
    excitePulseQ1 = RectSine(settings_ex[awg_ex_Q_ch]['A'], 1./settings_ex[awg_ex_Q_ch]['F'], excite_pulse_length, 
                             settings_ex[awg_ex_Q_ch]['P'], settings_ex[awg_ex_Q_ch]['O'], \
                             max_delay_with_pulses-excite_delay-excite_pulse_length*4, sample_period = 1./SampleRate_ex)
    excitePulseI2 = RectSine(settings_ex[awg_ex_I_ch]['A'], 1./settings_ex[awg_ex_I_ch]['F'], excite_pulse_length*2, \
                             2*np.pi*(settings_ex[awg_ex_I_ch]['F'])*(excite_delay/2+excite_pulse_length), \
                             0, \
                             max_delay_with_pulses-excite_delay/2-excite_pulse_length*3, sample_period = 1./SampleRate_ex)
    excitePulseQ2 = RectSine(settings_ex[awg_ex_Q_ch]['A'], 1./settings_ex[awg_ex_Q_ch]['F'], excite_pulse_length*2, 
                             settings_ex[awg_ex_Q_ch]['P']+\
                             2*np.pi*(settings_ex[awg_ex_I_ch]['F'])*(excite_delay/2+excite_pulse_length), \
                             0, \
                             max_delay_with_pulses-excite_delay/2-excite_pulse_length*3, sample_period = 1./SampleRate_ex)
    excitePulseI3 = RectSine(settings_ex[awg_ex_I_ch]['A'], 1./settings_ex[awg_ex_I_ch]['F'], excite_pulse_length, \
                             2*np.pi*(settings_ex[awg_ex_I_ch]['F'])*(excite_delay+3*excite_pulse_length), \
                             0, \
                             max_delay_with_pulses-excite_pulse_length, sample_period = 1./SampleRate_ex)
    excitePulseQ3 = RectSine(settings_ex[awg_ex_Q_ch]['A'], 1./settings_ex[awg_ex_Q_ch]['F'], excite_pulse_length, 
                             settings_ex[awg_ex_Q_ch]['P']+\
                             2*np.pi*(settings_ex[awg_ex_I_ch]['F'])*(excite_delay+3*excite_pulse_length), \
                             0, \
                             max_delay_with_pulses-excite_pulse_length, sample_period = 1./SampleRate_ex)
    excitePulseI = excitePulseI1 + excitePulseI2 + excitePulseI3
    excitePulseQ = excitePulseQ1 + excitePulseQ2 + excitePulseQ3
    ROPulseI = RectSine(settings_ro[awg_ro_I_ch]['A'], 1./settings_ro[awg_ro_I_ch]['F'], ro_pulse_length, 0, 
                        settings_ro[awg_ro_I_ch]['O'], max_delay_with_pulses, sample_period = 1./SampleRate_ro)
    ROPulseQ = RectSine(settings_ro[awg_ro_Q_ch]['A'], 1./settings_ro[awg_ro_Q_ch]['F'], ro_pulse_length, 
                        settings_ro[awg_ro_Q_ch]['P'], settings_ro[awg_ro_Q_ch]['O'], max_delay_with_pulses, sample_period = 1./SampleRate_ro)
    # setting trigger to start readout after excitation pulses
    triggerPulse = [0, zero_delay+int(max_delay_with_pulses/1e-9), 3, 100, 3, 1, 0, 4]
    SyncPulseProg = awg.intToBytes(triggerPulse)
    awg.ProgPulse(0, SyncPulseProg)
    awg.LoadRAM(awg_ex_I_ch, excitePulseI)
    awg.LoadRAM(awg_ex_Q_ch, excitePulseQ)
    awg.LoadRAM(awg_ro_I_ch, ROPulseI)
    awg.LoadRAM(awg_ro_Q_ch, ROPulseQ)
    #plt.figure(3)
    #plt.plot(excitePulseI)
    #plt.plot(excitePulseQ)
    
    daq0.stop()
    daq0.init_channel0_multiple_recording(nums=nums, segsize=hardware_segsize, posttrigger=posttrigger, amp=amp, offs=offs)
    daq0.buffer_setup()
    for software_average_id in range(software_averages):
        daq0.trigger_termination_50Ohm()
        daq0.start_with_trigger_and_waitready()
        data = np.sum(np.reshape(daq0.readout_singlechannel_multimode_bin(), (nums, binned_segsize, software_binning)), axis=2).T
        data_fft = np.fft.fft(data, axis=0)
        sample[:,delay_id] = sample[:,delay_id] + np.sum(data, axis=1)
        #covariation_matrix[:,:,delay_id] += np.einsum('ij,kj->ik', data, np.conj(data))
        #pickle.dump(data_fft[center_freq_point-8:center_freq_point+8, :], ro_results_output)
        S21[delay_id] = S21[delay_id] + np.sum(data_fft, axis=1)[center_freq_point]
        #plt.cla()
        #plt.plot(sample[:, pulse_length_id])
        #plt.pause(0.1)
        #print 'acq time: {0:5.3g}, ro time: {1:5.3g}, fft time: {2:5.3g}, mean time: {3:5.3g}, covariation time{4:5.3g}'.format(
        #    ro_start-acq_start, fft_start-ro_start, mean_start - fft_start, covariation_start - mean_start, cycle_end-covariation_start)
        daq0.stop()
    plt.figure(1)
    plt.cla()
    plt.plot(delays[:delay_id+1], np.abs(np.fft.fft(sample, axis=0)[center_freq_point,:delay_id+1]))
    plt.pause(0.1)
    plt.figure(2)
    plt.cla()
    plt.plot(delays[:delay_id+1], np.angle(np.fft.fft(sample, axis=0)[center_freq_point,:delay_id+1]))
    plt.pause(0.1)
    last_point = delay_id
#ro_results_output.close()

C:\Python27\Lib\site-packages\matplotlib\backend_bases.py:2399: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


KeyboardInterrupt: 

In [46]:
# фитует результаты измерений экспонентой
positive_delays =  delays[delays>0]
positive_delays_S21 = S21[delays>0]
f = lambda x,A0r,Aoi,A1r,A1i,t: (A0r+A0i*1j+((A1r-A0r)+(A1i-A0i)*1j)*np.exp(-x/t))
cost = lambda p: np.append((f(positive_delays,*p)-positive_delays_S21).real, (f(positive_delays,*p)-positive_delays_S21).imag)
# изначальные параметры:
A0r = positive_delays_S21[-1].real
A0i = positive_delays_S21[-1].imag
A1r = positive_delays_S21[0].real
A1i = positive_delays_S21[0].imag
d = (A1r-A0r)+1j*(A1i-A0i)
S21_PC = d.real*positive_delays_S21.real+d.imag*positive_delays_S21.imag
S21_PC=S21_PC-np.mean(S21_PC[-5:])
plt.figure()
plt.plot(positive_delays, S21_PC)
#print(np.sum(S21_PC), np.max(S21_PC), np.mean(np.diff(positive_delays)))
t = np.sum(S21_PC)/np.max(S21_PC)*np.mean(np.diff(positive_delays))
print((A0r,A0i,A1r,A1i,t), np.sum(np.abs(cost((A0r,A0i,A1r,A1i,t)))))
from scipy.optimize import leastsq
fitresults = leastsq(cost, (A0r,A0i,A1r,A1i,t))
print(fitresults[0], np.sum(np.abs(cost(fitresults[0]))))
fitted_curve = (positive_delays, np.log10(np.abs(f(positive_delays, *(fitresults[0]))))*10, np.angle(f(positive_delays, *(fitresults[0]))))
#print (fitresults)

((-119884526.71548374, 180651125.18256304, -133096170.88311888, 179689148.54410118, -9.078917083103382e-08), 5.3563807203267953e+19)
(array([  4.03434283e+09,   1.80651125e+08,  -1.17122567e+08,
         1.80666835e+08,  -4.05084289e-03]), 251824174.76599681)


In [61]:
execfile('save_pkl.py')
time_domain_box_setup = {'use_coupler':True, 'excite_lo':'lo1', 'ro_lo':'pna', 
                         'excite_lo_freq':excite_lo_freq, 'ro_lo_freq':ro_lo_freq, }
dac_setup = {'samplerate': samplerate, 'zero_delay': zero_delay, 'sample_size': hardware_segsize, 'posttrigger': posttrigger,
          'amp': 50, 'offs': 0,
          'hardware_averages': nums, 'software_averages': software_averages,}
header = {'date':datetime.datetime.now(), 'type': 'Spin echo', 'name': 'first', 'pulse_settings': settings, 
          'ro_pulse_length':ro_pulse_length, 'excite_pulse_length':excite_pulse_length,
          'dac_setup':dac_setup, 'time_domain_box_setup':time_domain_box_setup}
annotation='Decay time: T2*={0:3.2f} us'.format(fitresults[0][4]*1e6)
measurement = ((delays, np.asarray([0]), np.log10(np.abs(S21))*10, np.angle(S21)))
save_pkl(header, measurement, curve_fit=fitted_curve, annotation=annotation)

# Rabi power dependance

In [81]:
#Длительность 1 возбуждающего импульса красивой формы
pulse_length = 8e-9
# Длительность всего набора импульсов
total_pulses_lengths = 240e-9
# Число импульсов в таком случае
pulse_num = int(total_pulses_lengths / pulse_length)

pulse_amplitudes = np.linspace(0, 1, 201)

PulsePeriod = 20e-6
ro_pulse_length = 2e-6
#Длительность выборки DAQ
SampleRate = 500e6
nums = 5000
Range = 50
software_averages = 2

lo1.set_frequency(ex_lo_freq)
pna.set_cw_freq(ro_lo_freq)

#Настраиваем awg
data_len = int(PulsePeriod/2e-9)+1 
awg.set_repetition_period(PulsePeriod)
awg_tek.set_repetition_period(PulsePeriod - 1e-6)

#Инициализация DAQ
signal_size = int(np.ceil(4./3.*(total_pulses_lengths+ro_pulse_length+500e-9)*SampleRate))
segsize = 2*int( 2**np.ceil(np.log2(signal_size)) )
print segsize
posttrigger = segsize*3/4
signal_start = segsize - posttrigger 
signal_end = segsize - posttrigger + signal_size + zero_delay*2
if signal_end>segsize:
    signal_end = segsize
#segsize = 2048
#posttrigger = 512*3
    
daq0.stop()
daq0.set_spc_samplerate(SampleRate)
daq0.init_channel01_multiple_recording(rate=SampleRate, nums=nums, segsize=segsize, 
                                  posttrigger=posttrigger, amp0=Range, amp1=Range)
daq0.trigger_termination_50Ohm()
daq0.buffer_setup()


lo1.set_status(1)
#segsize = signal_end -  signal_start
#Контейнеры для данных
measQ = np.zeros((len(pulse_amplitudes), segsize))
measI = np.zeros((len(pulse_amplitudes), segsize))
I_Rabi = np.zeros( len(pulse_amplitudes) )
Q_Rabi = np.zeros( len(pulse_amplitudes) )

#Функции для демодуляции
t = linspace(0.,segsize/SampleRate, segsize, endpoint=False)
dm_sin = sin(2.*pi*ro_if*t)
dm_cos = cos(2.*pi*ro_if*t)

#Графики
plt.figure("I_Rabi")
amp_plot = plt.plot(pulse_amplitudes, I_Rabi)[0]
plt.figure("Q_Rabi")
ph_plot = plt.plot(pulse_amplitudes, Q_Rabi)[0]
awg_ex.set_trigger_mode('TRIG')

max_pulse_length = pulse_length

for i, pulse_amplitude in enumerate(pulse_amplitudes):
    #Чтобы хорошо разрешить эффект, давайте бахнем не один импульс, а сразу много! Скажем, штук 100.
    pulse_x_I = np.zeros (data_len*2)
    pulse_x_Q = np.zeros (data_len*2)
    for j in range (pulse_num):
        pulse_shape_x_I = gauss_rect_sine(1/ex_if*SampleRate_ex, pulse_length*SampleRate_ex, pulse_length*j*ex_if*2*np.pi, 2, 2)
        pulse_shape_x_Q = gauss_rect_sine(1/ex_if*SampleRate_ex, pulse_length*SampleRate_ex, pulse_length*j*ex_if*2*np.pi+settings_ex[awg_ex_Q_ch]['P'], 2, 2)
        pulse_x_I[j*pulse_shape_x_I.size+1:(j+1)*pulse_shape_x_I.size+1] = \
                        pulse_amplitude*settings_ex[awg_ex_I_ch]['A']*pulse_shape_x_I + settings_ex[awg_ex_I_ch]['O']
        pulse_x_Q[j*pulse_shape_x_Q.size+1:(j+1)*pulse_shape_x_Q.size+1] =\
                        pulse_amplitude*settings_ex[awg_ex_Q_ch]['A']*pulse_shape_x_Q + settings_ex[awg_ex_Q_ch]['O']
    pulse_x_I [0] = settings_ex[awg_ex_I_ch]['O']
    pulse_x_Q [0] = settings_ex[awg_ex_Q_ch]['O']
    pulse_x_I [pulse_num*pulse_shape_x_I.size+1:data_len*2] = settings_ex[awg_ex_I_ch]['O']
    pulse_x_Q [pulse_num*pulse_shape_x_Q.size+1:data_len*2] = settings_ex[awg_ex_Q_ch]['O']
    #exPulseI = RectSine(settings_ex[awg_ex_I_ch]['A'], 1./ex_if, pulse_length, 
    #                    settings_ex[awg_ex_I_ch]['P'], 
    #                    settings_ex[awg_ex_I_ch]['O'], 0)
    #exPulseQ = RectSine(settings_ex[awg_ex_Q_ch]['A'], 1./ex_if,
    #                    pulse_length, settings_ex[awg_ex_Q_ch]['P'], settings_ex[awg_ex_Q_ch]['O'],
    #                    0)


    #exPulseI[0] = settings_ex[awg_ex_I_ch]['O']
    #exPulseQ[0] = settings_ex[awg_ex_Q_ch]['O']
    #Программируем импульсы в awg
    awg_ex.LoadRAM(awg_ex_I_ch, pulse_x_I[0:data_len*2])
    awg_ex.LoadRAM(awg_ex_Q_ch, pulse_x_Q[0:data_len*2])
    awg_tek.run()
    #Считывающий импульс сразу после возбуждающего
    roPulseI = RectSine(settings_ro[awg_ro_I_ch]['A'], 1./ro_if, ro_pulse_length,
                    settings_ro[awg_ro_I_ch]['P'], settings_ro[awg_ro_I_ch]['O'], total_pulses_lengths+500e-9)
    roPulseQ = RectSine(settings_ro[awg_ro_Q_ch]['A'], 1./ro_if, ro_pulse_length, 
                    settings_ro[awg_ro_Q_ch]['P'], settings_ro[awg_ro_Q_ch]['O'], total_pulses_lengths+500e-9)
    awg_ro.LoadRAM(awg_ro_I_ch, roPulseI)
    awg_ro.LoadRAM(awg_ro_Q_ch, roPulseQ)
# setting trigger to start readout immideately after excitation puls
# zero_delay надо сначала померить в delay synchronization
#triggerPulse = [0, zero_delay+int(max_pulse_length/1e-9), 3, 100, 3, 1, 0, 4]
#SyncPulseProg = awg.intToBytes(triggerPulse)
#awg.ProgPulse(0, SyncPulseProg)
    
    for software_average_id in range(software_averages):
        
        #Мерием по триггеру
        daq0.start_with_trigger_and_waitready()
        #Усредняем результат
        measI[i] += np.sum(daq0.readout_doublechannel_multimode_bin()[0],
                           axis=0)
        measQ[i] += np.sum(daq0.readout_doublechannel_multimode_bin()[1],
                           axis=0)
        measI[i] -=np.mean(measI[i]) 
        measQ[i] -=np.mean(measQ[i])
        
        daq0.stop()
     
    #Делаем преобразованеи до нулевой частоты 
    I_Rabi[i] = np.mean(measI[i]*dm_cos - measQ[i]*dm_sin)/(nums*software_averages)
    Q_Rabi[i] = np.mean(measQ[i]*dm_cos + measI[i]*dm_sin)/(nums*software_averages)
      
    #Рисуем графики
    plt.figure("I_Rabi")
    amp_plot.set_data(pulse_amplitudes[:i+1], I_Rabi[:i+1])
    plt.ylim((np.min(I_Rabi[:i+1]), np.max(I_Rabi[:i+1])))
    plt.figure("Q_Rabi")
    ph_plot.set_data(pulse_amplitudes[:i+1], Q_Rabi[:i+1])
    plt.ylim((np.min(Q_Rabi[:i+1]), np.max(Q_Rabi[:i+1])))
    plt.pause(0.01)   


4096


C:\Python27\Lib\site-packages\matplotlib\axes\_base.py:2809: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=-0.383502744849, top=-0.383502744849
  'bottom=%s, top=%s') % (bottom, top))
C:\Python27\Lib\site-packages\matplotlib\axes\_base.py:2809: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=0.43912677976, top=0.43912677976
  'bottom=%s, top=%s') % (bottom, top))


KeyboardInterrupt: 

In [69]:
plt.figure()
plt.pcolormesh(measI)
plt.figure()
plt.pcolormesh(measQ)

In [83]:
from scipy import optimize

S21 = I_Rabi+1j*Q_Rabi
Rabi_fft=np.fft.fft(S21-np.mean(S21))
frequencies = np.fft.fftfreq(len(pulse_amplitudes), pulse_amplitudes[1]-pulse_amplitudes[0])
plt.figure('FFT')
plt.plot(frequencies, np.abs(Rabi_fft))
fR = frequencies[np.argmax(np.abs(Rabi_fft))]
print (fR)

fitfunc = lambda p, x: p[0]*np.cos(2*np.pi/p[1]*x+p[2])+p[3]
errfunc = lambda p, x, y: fitfunc(p, x) - y 
p0 = [0.003, 1/fR, 0.,-0.014] 
p1, success = optimize.leastsq(errfunc, p0[:], args=(pulse_amplitudes, I_Rabi))
plt.figure("I_Rabi")
amp = np.linspace(pulse_amplitudes.min(), pulse_amplitudes.max(), 100)
plt.plot(pulse_amplitudes, I_Rabi, "bo", amp, fitfunc(p1, amp), "g-") # Plot of the data and the fit

20.7920792079


# Gaussian pulse amplitude

In [104]:
pulse_amp = np.abs(p1[1]/4*61)
print(pulse_amp)

0.75258293681


In [107]:
pulse_amp = np.abs(p1[1]/4*61)
print(pulse_amp)

0.794630339643


In [110]:
pulse_amp = np.abs(p1[1]/4*61)
print(pulse_amp)

0.732033243184


In [114]:
pulse_amp = np.abs(p1[1]/4*61)
print(pulse_amp)

0.758177995038


In [117]:
pulse_amp = np.abs(p1[1]/4*61)
print(pulse_amp)

0.742412590291


In [120]:
pulse_amp = np.abs(p1[1]/4*61)
print(pulse_amp)

0.746781138021


In [80]:
# 16/4 gaussian pulses
pulse_amp_x = 0.6
pulse_amp_y = 0.6
pulse_amp_x_hd = 0.985
pulse_amp_y_hd = 0.985
# 32/6 gaussian pulses
#pulse_amp_x = 0.528
#pulse_amp_y = 0.498
# 24/6 gaussian pulses
pulse_amp_x_24 = 1.
pulse_amp_y_24 = 1.

In [81]:
PulsePeriod = 20e-6

In [89]:
from scipy.signal import gaussian, convolve

gauss_x =     {'ex': pulse_amp_x*gaussian(16, 3),    'ro': np.zeros(8)}
gauss_x_rev = {'ex': (-1)*pulse_amp_x*gaussian(16, 3),    'ro': np.zeros(8)}
gauss_y =     {'ex': pulse_amp_y*gaussian(16, 3)*1j, 'ro': np.zeros(8)}
gauss_y_rev =     {'ex': (-1)*pulse_amp_y*gaussian(16, 2)*1j, 'ro': np.zeros(8)}
gauss_x_24 =     {'ex': np.hstack([np.zeros(3), pulse_amp_x_24*gaussian(18, 4), np.zeros(3)]),    'ro': np.zeros(12)}
gauss_x_24_rev = {'ex': (-1)*np.hstack([np.zeros(3), pulse_amp_x_24*gaussian(18, 4), np.zeros(3)]),    'ro': np.zeros(12)}
gauss_y_24 =     {'ex': np.hstack([np.zeros(3), pulse_amp_x_24*gaussian(18, 4), np.zeros(3)])*1j, 'ro': np.zeros(12)}
gauss_y_24_rev = {'ex': (-1)*np.hstack([np.zeros(3), pulse_amp_x_24*gaussian(18, 4), np.zeros(3)])*1j,    'ro': np.zeros(12)}
ro      =     {'ex': np.zeros(2000),               'ro': np.ones(1000)}
pause   = lambda t: {'ex':np.zeros(2*t),       'ro': np.zeros(t)}

pulse_x = gauss_x_24
pulse_y = gauss_y_24
pulse_x_rev = gauss_x_24_rev
pulse_y_rev = gauss_y_24_rev

#gauss_x_h = np.hstack([[0], gauss_x['ex'], [0]])
#DRAG HD
#gauss_x_hd = np.diff(gauss_x_h)
#gauss_x_h1 = gauss_x_hd [1:]
#gauss_x_h2 = gauss_x_hd [:-1]
#gauss_x_hd = (gauss_x_h1 + gauss_x_h2)/2.*SampleRate_ro
#Anharmonicity = 200e6*2*np.pi
#gauss_x_HD =  {'ex': (gauss_x['ex']-13*1j*gauss_x_hd/(Anharmonicity*2))*pulse_amp_x_hd,    'ro':gauss_x['ro']}
#gauss_x_HD_rev =  {'ex': -(gauss_x['ex']-13*1j*gauss_x_hd/(Anharmonicity*2))*pulse_amp_x_hd,    'ro':gauss_x['ro']}

def seq2points(pulse_seq, num_points_ex, num_points_ro):
    channels = ['ex', 'ro']
    pulse_shape = {'ex':[0, 0]*8, 'ro':[0]*8}
    for channel in channels:
        for pulse in pulse_seq:
            pulse_shape[channel].extend(pulse[channel])
        pulse_shape[channel] = np.asarray(pulse_shape[channel])
        
    l_ex = pulse_shape['ex'].size
    l_ro = pulse_shape['ro'].size
    points_ex = np.linspace (0, l_ex-1, l_ex)
    points_ro = np.linspace (0, l_ro-1, l_ro)
    ex_I = settings_ex[awg_ex_I_ch]['O']*np.ones(num_points_ex)
    ex_Q = settings_ex[awg_ex_Q_ch]['O']*np.ones(num_points_ex)
    ro_I = settings_ro[awg_ro_I_ch]['O']*np.ones(num_points_ro)
    ro_Q = settings_ro[awg_ro_Q_ch]['O']*np.ones(num_points_ro)
    ex_I[:l_ex] = np.real(pulse_shape['ex']*settings_ex[awg_ex_I_ch]['A']*np.exp (1j*(2*np.pi*ex_if*points_ex/SampleRate_ex+settings_ex[awg_ex_I_ch]['P'])))+settings_ex[awg_ex_I_ch]['O']
    ex_Q[:l_ex] = np.real(pulse_shape['ex']*settings_ex[awg_ex_Q_ch]['A']*np.exp (1j*(2*np.pi*ex_if*points_ex/SampleRate_ex+settings_ex[awg_ex_Q_ch]['P'])))+settings_ex[awg_ex_Q_ch]['O']
    ro_I[:l_ro] = np.real(pulse_shape['ro']*settings_ro[awg_ro_I_ch]['A']*np.exp (1j*(2*np.pi*ro_if*points_ro/SampleRate_ro+settings_ro[awg_ro_I_ch]['P'])))+settings_ro[awg_ro_I_ch]['O']
    ro_Q[:l_ro] = np.real(pulse_shape['ro']*settings_ro[awg_ro_Q_ch]['A']*np.exp (1j*(2*np.pi*ro_if*points_ro/SampleRate_ro+settings_ro[awg_ro_Q_ch]['P'])))+settings_ro[awg_ro_Q_ch]['O']
    ro_daq = np.roll(convolve(pulse_shape['ro']>0.5, np.ones(800)/800, mode='full'), -200)>0.25
    
    last_daq_point = np.max(np.where(ro_daq>0))
    points_ro = np.linspace (0, last_daq_point-1, last_daq_point)
    
    ro_dm = np.exp (1j*(2*np.pi*ro_if*points_ro/SampleRate_ro))
    
    #plt.figure('ro I, Q, dm, daq')
    #plt.plot(ex_I)
    #plt.plot(ex_Q)
    #plt.plot(ro_dm*8000)
    #plt.plot(ro_daq*8000)
    
    return ex_I, ex_Q, ro_I, ro_Q, ro_daq, ro_dm


In [90]:
def sweep1D_mean_daq (daq, param1_vals, param1_setter, ro_dm, software_averages=1, plot=True):
    samples = np.zeros((len(param1_vals), ro_dm.size), dtype=np.complex)
    S21 = np.zeros(len(param1_vals), dtype=np.complex)
    
    #Графики
    if plot:
        plt.figure("I_sweep")
        I_plot = plt.plot(param1_vals, np.real(S21), marker='o')[0]
        plt.figure("Q_sweep")
        Q_plot = plt.plot(param1_vals, np.imag(S21), marker='o')[0]
    
    for i, param1_val in enumerate(param1_vals):
        param1_setter(param1_val)
        for software_average_id in range(software_averages):
        #Мерием по триггеру
            daq.start_with_trigger_and_waitready()
            #Усредняем результат
            samples[i,:] +=    np.mean(daq.readout_doublechannel_multimode_bin()[0], axis=0)[:ro_dm.size]/software_averages
            samples[i,:] += 1j*np.mean(daq.readout_doublechannel_multimode_bin()[1], axis=0)[:ro_dm.size]/software_averages 
            daq.stop()    
        #Делаем преобразованеи до нулевой частоты 
        S21[i] = np.mean(samples[i,:] * ro_dm)
      
        #Рисуем графики
        if plot:
            plt.figure("I_sweep")
            I_plot.set_data(param1_vals[:i+1], np.real(S21)[:i+1])
            plt.ylim((np.min(np.real(S21)[:i+1]), np.max(np.real(S21)[:i+1])))
            plt.figure("Q_sweep")
            Q_plot.set_data(param1_vals[:i+1], np.imag(S21)[:i+1])
            plt.ylim((np.min(np.imag(S21)[:i+1]), np.max(np.imag(S21)[:i+1])))
            plt.pause(0.01)   

    return S21, samples

In [91]:
def tomography_mean_daq (daq, nums, ro_dm, segsize, software_averages=1, plot_color='red', save_samples=True, classifier=None, plot=True, classifier_args=None):
    if save_samples:
        samples = np.zeros((nums*software_averages, ro_dm.size), dtype=np.complex)
    else:
        samples = np.zeros((nums, ro_dm.size), dtype=np.complex)
        S21 = np.zeros(nums*software_averages, dtype=np.complex)
    for software_average_id in range(software_averages):
    #Мерием по триггеру
        daq.start_with_trigger_and_waitready()
        #Усредняем результат
        if save_samples:
            samples[software_average_id*nums:(software_average_id+1)*nums,:] += daq.readout_doublechannel_multimode_bin()[0][:,:ro_dm.size]
            samples[software_average_id*nums:(software_average_id+1)*nums,:] += 1j*daq.readout_doublechannel_multimode_bin()[1][:,:ro_dm.size]
        else:
            samples[:,:] = daq.readout_doublechannel_multimode_bin()[0][:,:ro_dm.size]
            samples[:,:] += 1j*daq.readout_doublechannel_multimode_bin()[1][:,:ro_dm.size]
            if classifier:
                S21[software_average_id*nums:(software_average_id+1)*nums] = classifier(samples, **classifier_args)
            else:
                S21[software_average_id*nums:(software_average_id+1)*nums] = np.mean(samples * ro_dm, axis=1)
        daq.stop()    
    #Делаем преобразованеи до нулевой частоты 
    if save_samples:
        if classifier:
            S21 = classifier(samples, **classifier_args)
        else:
            S21 = np.mean(samples * ro_dm, axis=1)

    #Рисуем графики
    if plot:
        plt.scatter(np.real(S21),np.imag(S21),color=plot_color,s=1)
        plt.scatter(np.real(np.mean(S21)),np.imag(np.mean(S21)),color=plot_color,s=50, marker='^')
        plt.pause(0.01)   

    return S21, samples

In [92]:
def tomography_pulse_seq (pulse_seq, pulse_x, pulse_y, pulse_x_rev, pulse_y_rev):
#sigma_z
    pulse_seq_z = [i for i in pulse_seq]
    pulse_seq_z.append(pause(pulse_x['ro'].size))
    pulse_seq_z.append(pause(pulse_x['ro'].size))
    pulse_seq_z.append(pause(8))
    pulse_seq_z.append(pause(250))
    pulse_seq_z.append(ro)
#sigma_x
    pulse_seq_x = [i for i in pulse_seq]
    pulse_seq_x.append(pause(pulse_x['ro'].size))
    pulse_seq_x.extend ([pulse_y])
    pulse_seq_x.append(pause(8))
    pulse_seq_x.append(pause(250))
    pulse_seq_x.append(ro)
#sigma_y
    pulse_seq_y = [i for i in pulse_seq]
    pulse_seq_y.append(pause(pulse_x['ro'].size))
    pulse_seq_y.extend ([pulse_x_rev])
    pulse_seq_y.append(pause(8))
    pulse_seq_y.append(pause(250))
    pulse_seq_y.append(ro)
#sigma_x_rev
    pulse_seq_x_rev = [i for i in pulse_seq]
    pulse_seq_x_rev.append(pause(pulse_x['ro'].size))
    pulse_seq_x_rev.extend ([pulse_y_rev])
    pulse_seq_x_rev.append(pause(8))
    pulse_seq_x_rev.append(pause(250))
    pulse_seq_x_rev.append(ro)
#sigma_y_rev
    pulse_seq_y_rev = [i for i in pulse_seq]
    pulse_seq_y_rev.append(pause(pulse_x['ro'].size))
    pulse_seq_y_rev.extend ([pulse_x])
    pulse_seq_y_rev.append(pause(8))
    pulse_seq_y_rev.append(pause(250))
    pulse_seq_y_rev.append(ro)   
    
    return pulse_seq_z, pulse_seq_x, pulse_seq_y, pulse_seq_x_rev, pulse_seq_y_rev

In [93]:
def classifier(samples,clf_offset=None,clf_vector=None, ro_daq=None, ro_dm=None):
    samples_noavg = (samples.T-np.mean(samples,axis=1)).T
    samples_noavg = samples_noavg * ro_daq[:ro_dm.size]
    return np.mean((samples_noavg - clf_offset)*clf_vector, axis=1)

In [94]:
#this cell calibrates |0> & |1> states in samples space
def create_clf_samples (N,pulse_len):
    nums=500
    pulse_param = (0,1)
    time = (N+4)*pulse_len
    pulse_seq = [pause(time)]+[pause(8), pause(250), ro]
    ex_I, ex_Q, ro_I, ro_Q, ro_daq, ro_dm = seq2points(pulse_seq, PulsePeriod*SampleRate_ex, PulsePeriod*SampleRate_ro)
    awg_ex.set_repetition_period(PulsePeriod)
    awg_ro.set_repetition_period(PulsePeriod)

    #awg_ro.LoadRAM(awg_ro_I_ch, ro_I.astype(int).tolist())
    awg_ro.LoadRAM(awg_ro_Q_ch, ro_Q.astype(int).tolist())

    last_daq_point = np.max(np.where(ro_daq>0))
    segsize = 2*int( 2**np.ceil(np.log2(last_daq_point*4/3)) )
    posttrigger = segsize*3/4
    pretrigger = segsize-posttrigger

    ro_dm = np.hstack([np.zeros(pretrigger), ro_dm])
    ro_daq = np.hstack([np.zeros(pretrigger), ro_daq])

    amp = 50

    daq0.stop()
    daq0.set_spc_samplerate(SampleRate_ro)
    daq0.init_channel01_multiple_recording(rate=SampleRate_ro, nums=nums, segsize=segsize, 
                                      posttrigger=posttrigger, amp0=amp, amp1=amp)
    daq0.trigger_termination_50Ohm()
    daq0.buffer_setup()

    lo1.set_frequency(ex_lo_freq)
    pna.set_cw_freq(ro_lo_freq)
    lo1.set_status(1)
    def set_pulse (param):
        pulse_seq = [pause(N*pulse_len)]
        if param == 1:
            pulse_seq.extend([pulse_x]*2)
        else :
            pulse_seq.append (pause(pulse_len*2))
        pulse_seq_z, pulse_seq_x, pulse_seq_y, pulse_seq_x_rev, pulse_seq_y_rev = tomography_pulse_seq (pulse_seq, pulse_x, pulse_y, pulse_x_rev, pulse_y_rev)
        ex_I, ex_Q, ro_I, ro_Q, ne_nujno, toje_ne_nujno= seq2points(pulse_seq_z, PulsePeriod*SampleRate_ex, PulsePeriod*SampleRate_ro)
        awg_ex.LoadRAM(awg_ex_I_ch, ex_I.astype(int).tolist())
        awg_ex.LoadRAM(awg_ex_Q_ch, ex_Q.astype(int).tolist())
        awg_ro.LoadRAM(awg_ro_I_ch, ro_I.astype(int).tolist())
        awg_ro.LoadRAM(awg_ro_Q_ch, ro_Q.astype(int).tolist())
        awg_ex.run()    

    S21, samples = sweep1D_mean_daq (daq0,pulse_param, set_pulse, ro_dm, software_averages=100, plot=True)

    samples_noavg = (samples.T-np.mean(samples,axis=1)).T
    samples_noavg = samples_noavg * ro_daq[:ro_dm.size]
    clf_offset = np.mean(samples_noavg, axis=0)
    clf_vector = np.conj(np.diff(samples_noavg, axis=0))
    clf_vector = clf_vector/np.mean(np.abs(clf_vector)**2)
    proj = np.mean((samples_noavg - clf_offset)*clf_vector, axis=1)
    return clf_offset, clf_vector, ro_daq, ro_dm

In [95]:
#this cell for Rabi oscillations
N_max = 100
pulse=gauss_x
pulse_len = 8

pulse_seq = [pause(N_max*pulse_len)]+[pause(250), ro]
ex_I, ex_Q, ro_I, ro_Q, ro_daq, ro_dm = seq2points(pulse_seq, PulsePeriod*SampleRate_ex, PulsePeriod*SampleRate_ro)
awg_ro.LoadRAM(awg_ro_I_ch, ro_I.astype(int).tolist())
awg_ro.LoadRAM(awg_ro_Q_ch, ro_Q.astype(int).tolist())

awg_ex.set_repetition_period(PulsePeriod)
awg_ro.set_repetition_period(PulsePeriod)

# когда последняя точка которую надо записать?
last_daq_point = np.max(np.where(ro_daq>0))
segsize = 2*int( 2**np.ceil(np.log2(last_daq_point*4/3)) )
posttrigger = segsize*3/4
pretrigger = segsize-posttrigger

ro_dm = np.hstack([np.zeros(pretrigger), ro_dm])
ro_daq = np.hstack([np.zeros(pretrigger), ro_daq])

#Инициализация DAQ
amp = 50
nums = 500

daq0.stop()
daq0.set_spc_samplerate(SampleRate_ro)
daq0.init_channel01_multiple_recording(rate=SampleRate_ro, nums=nums, segsize=segsize, 
                                  posttrigger=posttrigger, amp0=amp, amp1=amp)
daq0.trigger_termination_50Ohm()
daq0.buffer_setup()

lo1.set_frequency(ex_lo_freq)
pna.set_cw_freq(ro_lo_freq)
lo1.set_status(1)

def set_pulse (N):
    pulse_seq = [pause((N_max-N)*pulse_len)]+[pulse]*N+[pause(250), ro]
    ex_I, ex_Q, ro_I, ro_Q, ro_daq, ro_dm = seq2points(pulse_seq, PulsePeriod*SampleRate_ex, PulsePeriod*SampleRate_ro)
    awg_ex.set_trigger_mode('TRIG')
    awg_ex.LoadRAM(awg_ex_I_ch, ex_I.astype(int).tolist())
    awg_ex.LoadRAM(awg_ex_Q_ch, ex_Q.astype(int).tolist())
    awg_ex.run()    

S21, samples = sweep1D_mean_daq (daq0, np.arange(0, 100), set_pulse, ro_dm, software_averages=4, plot=True)

C:\Python27\Lib\site-packages\matplotlib\axes\_base.py:2809: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=0.376554934471, top=0.376554934471
  'bottom=%s, top=%s') % (bottom, top))
C:\Python27\Lib\site-packages\matplotlib\axes\_base.py:2809: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=-0.392346085524, top=-0.392346085524
  'bottom=%s, top=%s') % (bottom, top))


KeyboardInterrupt: 

In [43]:
#this cell for Rabi oscillations
N_max = 100
pulse=gauss_x_24
pulse_len = 12

pulse_seq = [pause(N_max*pulse_len*2)]+[pause(250), ro]
ex_I, ex_Q, ro_I, ro_Q, ro_daq, ro_dm = seq2points(pulse_seq, PulsePeriod*SampleRate_ex, PulsePeriod*SampleRate_ro)
awg_ro.LoadRAM(awg_ro_I_ch, ro_I.astype(int).tolist())
awg_ro.LoadRAM(awg_ro_Q_ch, ro_Q.astype(int).tolist())

awg_ex.set_repetition_period(PulsePeriod)
awg_ro.set_repetition_period(PulsePeriod)

# когда последняя точка которую надо записать?
last_daq_point = np.max(np.where(ro_daq>0))
segsize = 2*int( 2**np.ceil(np.log2(last_daq_point*4/3)) )
posttrigger = segsize*3/4
pretrigger = segsize-posttrigger

ro_dm = np.hstack([np.zeros(pretrigger), ro_dm])
ro_daq = np.hstack([np.zeros(pretrigger), ro_daq])

#Инициализация DAQ
amp = 50
nums = 500

daq0.stop()
daq0.set_spc_samplerate(SampleRate_ro)
daq0.init_channel01_multiple_recording(rate=SampleRate_ro, nums=nums, segsize=segsize, 
                                  posttrigger=posttrigger, amp0=amp, amp1=amp)
daq0.trigger_termination_50Ohm()
daq0.buffer_setup()

lo1.set_frequency(ex_lo_freq)
pna.set_cw_freq(ro_lo_freq)
lo1.set_status(1)

def set_pulse (N):
    pulse_seq = [pause((N_max-N)*pulse_len*2)]+[gauss_x, pause(pulse_len)]*N+[pause(250), ro]
    ex_I, ex_Q, ro_I, ro_Q, ro_daq, ro_dm = seq2points(pulse_seq, PulsePeriod*SampleRate_ex, PulsePeriod*SampleRate_ro)
    awg_ex.set_trigger_mode('TRIG')
    awg_ex.LoadRAM(awg_ex_I_ch, ex_I.astype(int).tolist())
    awg_ex.LoadRAM(awg_ex_Q_ch, ex_Q.astype(int).tolist())
    awg_ex.run()    

S21, samples = sweep1D_mean_daq (daq0, np.arange(0, 100), set_pulse, ro_dm, software_averages=4, plot=True)

C:\Python27\Lib\site-packages\matplotlib\axes\_base.py:2809: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=0.16738816253, top=0.16738816253
  'bottom=%s, top=%s') % (bottom, top))
C:\Python27\Lib\site-packages\matplotlib\axes\_base.py:2809: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=-0.221620553235, top=-0.221620553235
  'bottom=%s, top=%s') % (bottom, top))


In [108]:
#this cell for tomography
pulse_len = 12
N = 2
time = (N+2)*pulse_len
pulse_seq = [pause(time)]+[pause(8), pause(250), ro]
ex_I, ex_Q, ro_I, ro_Q, ro_daq, ro_dm = seq2points(pulse_seq, PulsePeriod*SampleRate_ex, PulsePeriod*SampleRate_ro)
awg_ex.set_repetition_period(PulsePeriod)
awg_ro.set_repetition_period(PulsePeriod)

# когда последняя точка которую надо записать?
last_daq_point = np.max(np.where(ro_daq>0))
segsize = 2*int( 2**np.ceil(np.log2(last_daq_point*4/3)) )
posttrigger = segsize*3/4
pretrigger = segsize-posttrigger

ro_dm = np.hstack([np.zeros(pretrigger), ro_dm])
ro_daq = np.hstack([np.zeros(pretrigger), ro_daq])


#Инициализация DAQ

amp = 50
nums = 500
    
daq0.stop()
daq0.set_spc_samplerate(SampleRate_ro)
daq0.init_channel01_multiple_recording(rate=SampleRate_ro, nums=nums, segsize=segsize, 
                                  posttrigger=posttrigger, amp0=amp, amp1=amp)
daq0.trigger_termination_50Ohm()
daq0.buffer_setup()

lo1.set_frequency(ex_lo_freq)
pna.set_cw_freq(ro_lo_freq)
lo1.set_status(1)

pulse_seq = []

pulse_seq.extend([pulse_x]*2)
#pulse_seq.extend([pulse_x, pause(12)])
#pulse_seq.append(pause(48))
pulse_seq_z, pulse_seq_x, pulse_seq_y, pulse_seq_x_rev, pulse_seq_y_rev = tomography_pulse_seq (pulse_seq, pulse_x, pulse_y, pulse_x_rev, pulse_y_rev)
pulse_SEQ = []
pulse_SEQ.append (pulse_seq_z)
pulse_SEQ.append (pulse_seq_x)
pulse_SEQ.append (pulse_seq_y)
pulse_SEQ.append (pulse_seq_x_rev)
pulse_SEQ.append (pulse_seq_y_rev)

plot_colors = ['black', 'green', 'blue', 'red', 'purple']
tomo_proj = []
clf_offset,clf_vector, ro_daq, ro_dm = create_clf_samples (N,pulse_len)

plt.figure('IQ')
for meas_pulse_seq, plot_color in zip(pulse_SEQ, plot_colors):
    print (plot_color)
    ex_I, ex_Q, ro_I, ro_Q, ne_nujno, toje_ne_nujno= seq2points(meas_pulse_seq, PulsePeriod*SampleRate_ex, PulsePeriod*SampleRate_ro)
    awg_ex.LoadRAM(awg_ex_I_ch, ex_I.astype(int).tolist())
    awg_ex.LoadRAM(awg_ex_Q_ch, ex_Q.astype(int).tolist())
    awg_ro.LoadRAM(awg_ro_I_ch, ro_I.astype(int).tolist())
    awg_ro.LoadRAM(awg_ro_Q_ch, ro_Q.astype(int).tolist())
    awg_ex.set_trigger_mode('TRIG')
    awg_ex.run()    
    S21, samples = tomography_mean_daq (daq0, nums, ro_dm, segsize, software_averages=200,plot_color=plot_color,save_samples=False)
    tomo_proj.append(classifier(samples,clf_offset,clf_vector, ro_daq, ro_dm))
    
plt.figure()
plt.hist(tomo_proj, color=plot_colors)


black


KeyboardInterrupt: 

In [124]:
plt.plot(ro_daq[:ro_dm.size]*ro_dm)
plt.plot(np.mean(samples, axis=0))

In [289]:
#this cell for Rabi oscillations with tomography
N_max = 8
mean_tomo_proj = np.zeros((5,N_max))

pulse_x = gauss_x
pulse_y = gauss_y
pulse_x_rev = gauss_x_rev
pulse_y_rev = gauss_y_rev

for N in range (0,N_max):
    pulse_len = 8
    time = (N*2+4)*pulse_len
    pulse_seq = [pause(time)]+[pause(8), pause(250), ro]
    ex_I, ex_Q, ro_I, ro_Q, ro_daq, ro_dm = seq2points(pulse_seq, PulsePeriod*SampleRate_ex, PulsePeriod*SampleRate_ro)
    awg_ex.set_repetition_period(PulsePeriod)
    awg_ro.set_repetition_period(PulsePeriod)

    # когда последняя точка которую надо записать?
    last_daq_point = np.max(np.where(ro_daq>0))
    segsize = 2*int( 2**np.ceil(np.log2(last_daq_point*4/3)) )
    posttrigger = segsize*3/4
    pretrigger = segsize-posttrigger

    ro_dm = np.hstack([np.zeros(pretrigger), ro_dm])
    ro_daq = np.hstack([np.zeros(pretrigger), ro_daq])


    #Инициализация DAQ

    amp = 50
    nums = 500

    daq0.stop()
    daq0.set_spc_samplerate(SampleRate_ro)
    daq0.init_channel01_multiple_recording(rate=SampleRate_ro, nums=nums, segsize=segsize, 
                                      posttrigger=posttrigger, amp0=amp, amp1=amp)
    daq0.trigger_termination_50Ohm()
    daq0.buffer_setup()

    lo1.set_frequency(ex_lo_freq)
    pna.set_cw_freq(ro_lo_freq)
    lo1.set_status(1)

    pulse_seq = [pause(pulse_len*2)]

    pulse_seq.extend([pulse_x, pause(pulse_len)]*N)
    #pulse_seq.extend([pulse_x, pause(12)])
    #pulse_seq.append(pause(48))
    pulse_seq_z, pulse_seq_x, pulse_seq_y, pulse_seq_x_rev, pulse_seq_y_rev = tomography_pulse_seq (pulse_seq, pulse_x, pulse_y, pulse_x_rev, pulse_y_rev)
    pulse_SEQ = []
    pulse_SEQ.append (pulse_seq_z)
    pulse_SEQ.append (pulse_seq_x)
    pulse_SEQ.append (pulse_seq_y)
    pulse_SEQ.append (pulse_seq_x_rev)
    pulse_SEQ.append (pulse_seq_y_rev)

    plot_colors = ['black', 'green', 'blue', 'red', 'purple']
    tomo_proj = []
    clf_offset,clf_vector, ro_daq, ro_dm = create_clf_samples (N,pulse_len)
    #plt.figure('samples')
    #plt.plot(clf_offset, marker='o')
    #plt.figure('empty')

    #plt.figure('IQ')
    for meas_pulse_seq, plot_color in zip(pulse_SEQ, plot_colors):
        print (plot_color)
        ex_I, ex_Q, ro_I, ro_Q, ne_nujno, toje_ne_nujno= seq2points(meas_pulse_seq, PulsePeriod*SampleRate_ex, PulsePeriod*SampleRate_ro)
        awg_ex.LoadRAM(awg_ex_I_ch, ex_I.astype(int).tolist())
        awg_ex.LoadRAM(awg_ex_Q_ch, ex_Q.astype(int).tolist())
        awg_ro.LoadRAM(awg_ro_I_ch, ro_I.astype(int).tolist())
        awg_ro.LoadRAM(awg_ro_Q_ch, ro_Q.astype(int).tolist())
        plt.clf()
        awg_ex.set_trigger_mode('TRIG')
        awg_ex.run()    
        S21, samples = tomography_mean_daq (daq0, nums, ro_dm, segsize, software_averages=10,plot_color=plot_color,save_samples=False)
        tomo_proj.append(classifier(samples,clf_offset,clf_vector, ro_daq, ro_dm))
        #plt.figure('samples')
        #plt.plot(np.mean(samples,axis=0))
        #plt.figure('empty')
   
    mean_tomo_proj[:,N] = np.real(np.mean(tomo_proj, axis=1))
    print (mean_tomo_proj[:,N])


black
green
blue
red
purple
[-0.45930178  0.06071166  0.03250563  0.00557995  0.09101557]

C:\Python27\Lib\site-packages\matplotlib\axes\_base.py:2809: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=0.381124317352, top=0.381124317352
  'bottom=%s, top=%s') % (bottom, top))
C:\Python27\Lib\site-packages\matplotlib\axes\_base.py:2809: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=-0.478385688173, top=-0.478385688173
  'bottom=%s, top=%s') % (bottom, top))
C:\Python27\Lib\site-packages\matplotlib\axes\_base.py:2809: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=0.340548000326, top=0.340548000326
  'bottom=%s, top=%s') % (bottom, top))



black
green
blue
red
purple
[ 0.00995525  0.57080439  0.12386409 -0.46691015  0.03646404]

C:\Python27\Lib\site-packages\matplotlib\axes\_base.py:2809: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=-0.48197985618, top=-0.48197985618
  'bottom=%s, top=%s') % (bottom, top))
C:\Python27\Lib\site-packages\matplotlib\axes\_base.py:2809: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=0.379789575452, top=0.379789575452
  'bottom=%s, top=%s') % (bottom, top))



black
green
blue
red
purple
[ 0.20065102  0.09959081  0.47846442  0.03679767 -0.38747811]

...

C:\Python27\Lib\site-packages\matplotlib\axes\_base.py:2809: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=-0.493817135916, top=-0.493817135916
  'bottom=%s, top=%s') % (bottom, top))
C:\Python27\Lib\site-packages\matplotlib\axes\_base.py:2809: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=0.278732497471, top=0.278732497471
  'bottom=%s, top=%s') % (bottom, top))



black
green
blue
red
purple
[-0.18627439 -0.29904287  0.33976884  0.17330764 -0.46562133]

C:\Python27\Lib\site-packages\matplotlib\axes\_base.py:2809: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=-0.487156545362, top=-0.487156545362
  'bottom=%s, top=%s') % (bottom, top))
C:\Python27\Lib\site-packages\matplotlib\axes\_base.py:2809: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=0.30651368597, top=0.30651368597
  'bottom=%s, top=%s') % (bottom, top))



black
green
blue
red
purple
[-0.44817959 -0.04750015 -0.02471697 -0.00845929 -0.15601102]


C:\Python27\Lib\site-packages\matplotlib\axes\_base.py:2809: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=-0.482908108573, top=-0.482908108573
  'bottom=%s, top=%s') % (bottom, top))


In [2]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(mean_tomo_proj[0,:], mean_tomo_proj[1,:], mean_tomo_proj[2,:], c='r', marker='o')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_xlim([-0.6, 0.6])
ax.set_ylim([-0.6, 0.6])
ax.set_zlim([-0.6, 0.6])

NameError: name 'mean_tomo_proj' is not defined

In [141]:
#this cell calibrates |0> & |1> states in samples space
pulse_param = (0,1)
pulse_len = 12
N=4
time = N*pulse_len
pulse_seq = [pause(time)]+[pause(8), pause(250), ro]
ex_I, ex_Q, ro_I, ro_Q, ro_daq, ro_dm = seq2points(pulse_seq, PulsePeriod*SampleRate_ex, PulsePeriod*SampleRate_ro)
awg_ex.set_repetition_period(PulsePeriod)
awg_ro.set_repetition_period(PulsePeriod)

awg_ro.LoadRAM(awg_ro_I_ch, ro_I.astype(int).tolist())
awg_ro.LoadRAM(awg_ro_Q_ch, ro_Q.astype(int).tolist())

last_daq_point = np.max(np.where(ro_daq>0))
segsize = 2*int( 2**np.ceil(np.log2(last_daq_point*4/3)) )
posttrigger = segsize*3/4
pretrigger = segsize-posttrigger

ro_dm = np.hstack([np.zeros(pretrigger), ro_dm])
ro_daq = np.hstack([np.zeros(pretrigger), ro_daq])

amp = 50
nums = 500
    
daq0.stop()
daq0.set_spc_samplerate(SampleRate_ro)
daq0.init_channel01_multiple_recording(rate=SampleRate_ro, nums=nums, segsize=segsize, 
                                  posttrigger=posttrigger, amp0=amp, amp1=amp)
daq0.trigger_termination_50Ohm()
daq0.buffer_setup()

lo1.set_frequency(ex_lo_freq)
pna.set_cw_freq(ro_lo_freq)
lo1.set_status(1)
class_colors = ['black', 'red']
classes = [0, 1]


for plot_color, param in zip(class_colors, classes):
    pulse_seq = []
    if param == 1:
        pulse_seq.extend([pulse_x]*2)
    else :
        pulse_seq.append (pause(24))
    pulse_seq_z, pulse_seq_x, pulse_seq_y, pulse_seq_x_rev, pulse_seq_y_rev = tomography_pulse_seq (pulse_seq, pulse_x, pulse_y, pulse_x_rev, pulse_y_rev)
    ex_I, ex_Q, ro_I, ro_Q, ne_nujno, toje_ne_nujno= seq2points(pulse_seq_z, PulsePeriod*SampleRate_ex, PulsePeriod*SampleRate_ro)
    awg_ex.LoadRAM(awg_ex_I_ch, ex_I.astype(int).tolist())
    awg_ex.LoadRAM(awg_ex_Q_ch, ex_Q.astype(int).tolist())
    awg_ro.LoadRAM(awg_ro_I_ch, ro_I.astype(int).tolist())
    awg_ro.LoadRAM(awg_ro_Q_ch, ro_Q.astype(int).tolist())
    awg_ex.set_trigger_mode('TRIG')
    awg_ex.run()    
    
    plt.figure('IQ')
    S21, samples = tomography_mean_daq (daq0, nums, ro_dm, segsize, plot_color=plot_color,save_samples=False,classifier=classifier, software_averages=100, \
                                        classifier_args={'clf_offset':clf_offset,'clf_vector':clf_vector, 'ro_daq':ro_daq, 'ro_dm':ro_dm})
    classifier_results.append(np.real(S21))

average_text = 'AVG |0>: {0: 4.2f}, AVG |1>: {1:4.2f}'.format(np.mean(classifier_results[0]), np.mean(classifier_results[1]))
std_text = 'STD |0>: {0: 4.2f}, STD |1>: {1:4.2f}'.format(np.std(classifier_results[0]), np.std(classifier_results[1]))
q_text = 'q0(0): {0:4.2f}, q1(1): {1:4.2f}'.format(np.sqrt(np.mean(classifier_results[0]<0)), np.sqrt(np.mean(classifier_results[1]>0)))
Fidelity = (np.sqrt(np.mean(classifier_results[0]<0))+np.sqrt(np.mean(classifier_results[1]>0)))/2.
fidelity_text = 'Fidelity: {0: 4.2f}%'.format(Fidelity*100)
annotation = '\n'.join([average_text, std_text, q_text, fidelity_text])

plt.figure('clf')
plt.hist(classifier_results, label=classes)
plt.annotate(annotation, (0.3, 0.1), xycoords='axes fraction', \
             bbox={'alpha':1.0, 'pad':3, 'edgecolor':'black', 'facecolor':'white'})
plt.legend()

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-141-21bdea10a476> in <module>()
     52 
     53     plt.figure('IQ')
---> 54     S21, samples = tomography_mean_daq (daq0, nums, ro_dm, segsize, plot_color=plot_color,save_samples=False,classifier=classifier, software_averages=100,                                         classifier_args={'clf_offset':clf_offset,'clf_vector':clf_vector, 'ro_daq':ro_daq, 'ro_dm':ro_dm})
     55     classifier_results.append(np.real(S21))
     56 

<ipython-input-140-f385cd85f30b> in tomography_mean_daq(daq, nums, ro_dm, segsize, software_averages, plot_color, save_samples, classifier, plot, classifier_args)
     16             samples[:,:] += 1j*daq.readout_doublechannel_multimode_bin()[1][:,:ro_dm.size]
     17             if classifier:
---> 18                 S21[software_average_id*nums:(software_average_id+1)*nums] = classifier(sample

In [ ]:
num_pulses_range = np.arange(0, 20, 5).astype(int)
phi = np.arange(0, 2*np.pi+np.pi/8.,np.pi/8.)
pulse_len = 8
pulse_seq = [pause(pulse_len)]*(4*np.max(num_pulses_range)+4)+[pause(8), pause(250), ro]
ex_I, ex_Q, ro_I, ro_Q, ro_daq, ro_dm = seq2points(pulse_seq, PulsePeriod*SampleRate_ex, PulsePeriod*SampleRate_ro)
awg_ro.LoadRAM(awg_ro_I_ch, ro_I.astype(int).tolist())
awg_ro.LoadRAM(awg_ro_Q_ch, ro_Q.astype(int).tolist())

# когда последняя точка которую надо записать?
last_daq_point = np.max(np.where(ro_daq>0))

#Инициализация DAQ
segsize = 2*int( 2**np.ceil(np.log2(last_daq_point)) )
if ro_dm.size>segsize:
    ro_dm = ro_dm[:segsize]
else:
    ro_dm_new = np.zeros(segsize, dtype=np.complex)
    ro_dm_new[:ro_dm.size] = ro_dm
    ro_dm = ro_dm_new

print (segsize, ro_dm.size)
posttrigger = segsize*3/4
ro_dm = np.roll(ro_dm, -posttrigger)
amp = 50
nums = 1000
    
daq0.stop()
daq0.set_spc_samplerate(SampleRate_ro)
daq0.init_channel01_multiple_recording(rate=SampleRate_ro, nums=nums, segsize=segsize, 
                                  posttrigger=posttrigger, amp0=amp, amp1=amp)
daq0.trigger_termination_50Ohm()
daq0.buffer_setup()

lo1.set_frequency(ex_lo_freq)
pna.set_cw_freq(ro_lo_freq)
lo1.set_status(1)

def set_phase(phi):
    pulse_seq = [gauss_x_HD, pause(pulse_len*(4*np.max(num_pulses_range)-4*num_pulses+1))]
    gauss_rotated = gauss_x_HD.copy()
    gauss_rotated['ex'] = gauss_rotated['ex']*np.exp(1j*phi)
    pulse_seq.extend([gauss_x_HD, pause(pulse_len), gauss_x_HD_rev, pause(pulse_len)]*num_pulses)
    pulse_seq.extend([gauss_rotated])
    pulse_seq.append(pause(8))
    pulse_seq.append(ro)
    
    ex_I, ex_Q, ro_I, ro_Q, ro_daq, ro_dm = seq2points(pulse_seq, PulsePeriod*SampleRate_ex, PulsePeriod*SampleRate_ro)
    awg_ex.LoadRAM(awg_ex_I_ch, ex_I.astype(int).tolist())
    awg_ex.LoadRAM(awg_ex_Q_ch, ex_Q.astype(int).tolist())
    awg_ex.set_trigger_mode('TRIG')
    awg_ex.run()    

for num_pulses in num_pulses_range:
    S21, samples = sweep1D_mean_daq (daq0, phi, set_phase, ro_dm, software_averages=100)

In [ ]:
plt.plot(np.real(gauss_x_HD['ex']))
plt.plot(np.imag(gauss_x_HD['ex']))

In [ ]:
plt.figure()
plt.plot(np.real(S21))
plt.plot(np.imag(S21))

In [ ]:
plt.figure ('samples')
plt.pcolormesh (np.real(samples))
print S21.size, num_pulses_range.size

In [34]:
import mixer_calibration
imp.reload(mixer_calibration)
sa_added_measurer = mixer_calibration.sa_meas_added(pxa, lo1.set_status, ex_freq, ex_freq, nop=1, res_bw=1e6)
sa_added_measurer.set_precision(-110)
result = sa_added_measurer.measure_added()
print (sa_added_measurer.noise_floor, sa_added_measurer.precision)
print (result)

(array([-79.99654388]), array([-103.81034963]))
[-38.72156098]


In [ ]:
plt.plot(ro_daq)


In [ ]:
from scipy import optimize

fft=np.fft.fft(S21-np.mean(S21))
frequencies = np.fft.fftfreq(num_pulses_range.size, num_pulses_range[1]-num_pulses_range[0])
plt.figure('FFT')
plt.plot(frequencies, np.abs(fft))
fR = np.abs(frequencies[np.argmax(np.abs(fft))])
print (fR)
fitfunc = lambda p, x: p[0] + p[1]*np.exp(-x/p[2])*(np.sin(2*np.pi*fR*x + p[3]))
errfunc = lambda p, x, y: fitfunc(p, x) - y 
p0 = [1500, 1000, 60,0]
p1, success = optimize.leastsq(errfunc,p0, args=(num_pulses_range, S21))
plt.figure("S21")
amp = np.linspace(num_pulses_range.min(), num_pulses_range.max(), 100)
plt.plot(num_pulses_range, S21, "bo", amp, fitfunc(p1, amp), "g-") # Plot of the data and the fit

In [ ]:
execfile('save_pkl.py')
time_domain_box_setup = {'use_coupler':True, 'excite_lo':'lo1', 'ro_lo':'pna', 
                         'excite_lo_freq':excite_lo_freq, 'ro_lo_freq':ro_lo_freq, }
dac_setup = {'samplerate': samplerate, 'zero_delay': zero_delay, 'sample_size': segsize, 'posttrigger': posttrigger,
          'amp': amp, 'offs': offs,
          'hardware_averages': nums, 'software_averages': software_averages,}
header = {'date':datetime.datetime.now(), 'type': 'Rabi power dependance', 'name': '', 'pulse_settings': settings, 
          'ro_pulse_length':ro_pulse_length, 'excite_pulse_length':excite_pulse_length,
          'dac_setup':dac_setup, 'time_domain_box_setup':time_domain_box_setup}
sample_axis = np.linspace((posttrigger-hardware_segsize)/hardware_samplerate, posttrigger/hardware_samplerate, binned_segsize)
measurement =        ((pulse_lengths, excite_amplitudes[:last_amplitude_point+1],
                       np.log10(np.abs(S21[:,:last_amplitude_point +1]))*10, np.angle(S21[:,:last_amplitude_point +1])))
sample_measurement = ((np.linspace((posttrigger-hardware_segsize)/hardware_samplerate, posttrigger/hardware_samplerate, binned_segsize), 
                       pulse_lengths, excite_amplitudes[:last_amplitude_point +1],
                       sample[:,:,:last_amplitude_point+1], np.zeros(sample[:,:,:last_amplitude_point +1].shape)))
if compute_covariations:
    covariation_matrix_measurement = ((np.linspace((posttrigger-hardware_segsize)/hardware_samplerate, posttrigger/hardware_samplerate, binned_segsize), 
                       np.linspace((posttrigger-hardware_segsize)/hardware_samplerate, posttrigger/hardware_samplerate, binned_segsize), 
                       pulse_lengths, excite_amplitudes[:last_amplitude_point+1],
                       covariation_matrix[:,:,:,:last_amplitude_point+1], np.zeros(covariation_matrix[:,:,:,:last_amplitude_point+1].shape)))
    extended_covariation_matrix = np.zeros((binned_segsize+1, binned_segsize+1, len(pulse_lengths), last_amplitude_point+1))
    extended_covariation_matrix[0,0,:,:] = software_averages*nums
    extended_covariation_matrix[1:,0,:,:] = sample[:,:,:last_amplitude_point+1]
    extended_covariation_matrix[0,1:,:,:] = sample[:,:,:last_amplitude_point+1]
    extended_covariation_matrix[1:,1:,:,:] = covariation_matrix[:,:,:,:last_amplitude_point+1]
    extended_sample_axis = np.append(np.asarray([np.nan]), sample_axis)
header['type'] = 'Rabi 2D S21'
save_pkl(header, measurement)
header['type'] = 'Rabi 2D oscillogram'
save_pkl(header, sample_measurement)
if compute_covariations:
    header['type'] = 'Rabi 2D covariation matrix'
    save_pkl(header, covariation_matrix_measurement)
    header['type'] = 'Rabi 2D extended covariation matrix'
    save_pkl(header, (extended_sample_axis, extended_sample_axis, pulse_lengths, excite_amplitudes[:last_amplitude_point+1], 
                  extended_covariation_matrix, np.zeros(extended_covariation_matrix.shape)))

In [ ]:
plt.plot(np.abs(S21[:,:last_amplitude_point+1]))

In [ ]:
pulse_profile = gauss_rect_sine(8, 16, 0, 4, 2)
plt.plot(pulse_profile)

In [ ]:
pulse_shape_x_I = gauss_rect_sine(8, 16, 0, 4, 2)
pulse_shape_x_Q = gauss_rect_sine(8, 16, np.pi/2, 4, 2)
pulse_num = 83
pulse_x_I = np.zeros ((pulse_shape_x_I.size) * pulse_num +1)
pulse_x_Q = np.zeros ((pulse_shape_x_I.size) * pulse_num +1)
for i in range (pulse_num):
    pulse_x_I[i*pulse_shape_x_I.size+1:(i+1)*pulse_shape_x_I.size+1] = \
                        settings_ex[awg_ex_I_ch]['A']*pulse_shape_x_I + settings_ex[awg_ex_I_ch]['O']
    pulse_x_Q[i*pulse_shape_x_Q.size+1:(i+1)*pulse_shape_x_Q.size+1] =\
                        settings_ex[awg_ex_Q_ch]['A']*pulse_shape_x_Q + settings_ex[awg_ex_Q_ch]['O']
pulse_x_I [0] = settings_ex[awg_ex_I_ch]['O']
pulse_x_Q [0] = settings_ex[awg_ex_Q_ch]['O']
plt.plot(pulse_x_I)
plt.plot(pulse_x_Q)